# Data Processing
Obtain and Scrub

## Import libraries

In [ ]:
!pip install duckdb

In [ ]:

import pandas as pd
#data viz
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
#Google sheet
import gspread
from google.colab import auth
import google.auth # Import google.auth
#import duckdb
import duckdb


# Authenticate Google Cloud user
auth.authenticate_user()

# Get the authenticated credentials
credentials, project = google.auth.default()

# Authorize gspread with the obtained credentials
gc = gspread.authorize(credentials)

print("Authentication successful. 'gc' object is ready.")


Authentication successful. 'gc' object is ready.


## Functions

In [ ]:
def clean_currency(series: pd.Series) -> pd.Series:
    """
    Cleans currency-like strings such as '$232,562.00' or '1,200'
    and converts them to float.
    """
    return (
        series
        .astype(str)
        .str.replace(r"[^0-9.\-]", "", regex=True)   # remove $, commas, spaces, etc.
        .replace("", pd.NA)                         # empty string → NA
        .astype(float)
    )


In [ ]:
def print_outlier_message():
    print(f"\n🔍 Outlier threshold (99th percentile): ${outlier_threshold:,.0f}\n")

    if outlier_summary.empty:
        print("No outliers detected above the 99th percentile.\n")
    else:
        print("⚠️ Outliers removed from the following month-year groups:\n")
        for month, count in outlier_summary.items():
            print(f"  • {month.strftime('%b %Y')}: {count} sale(s)")
    print("\n")


In [ ]:
def print_agent_message():
  print("This analysis uses agent_id as the unique identifier for agents.The dataset contains homonymous agents (agents with the same first and last name), and the table specification explicitly states that agent_id is the unique key.")


## 🔹 Obtain - Reading Tables

In [ ]:
def read_sheet_as_df(sheet, worksheet_index: int) -> pd.DataFrame:
    """
    Reads a specific worksheet from a gspread sheet object and returns a clean Pandas DataFrame.

    Args:
        sheet: gspread Sheet object (returned by gc.open(...))
        worksheet_index: zero-based index of the worksheet to read

    Returns:
        pd.DataFrame with first row as header and remaining rows as data.
    """
    ws = sheet.get_worksheet(worksheet_index)
    rows = ws.get_all_values()

    if not rows:
        return pd.DataFrame()  # Return empty df if sheet is blank

    header, data = rows[0], rows[1:]
    return pd.DataFrame(data, columns=header)


# --- Open Google Sheet ---
sheet = gc.open("Data Analyst Trial - Data Set")

# --- Read Sheet 3 and Sheet 4 ---
df_sales = read_sheet_as_df(sheet, 1)
df_calls = read_sheet_as_df(sheet, 2)
df_agents = read_sheet_as_df(sheet, 3)
df_customers = read_sheet_as_df(sheet, 4)
df_markets = read_sheet_as_df(sheet, 5)


## 🔹 Scrub - Understanding the Data

### Sales DataFrame

In [ ]:
df_sales.head(3)

,sale_id,customer_id,sale_date,amount
0,1,2,1/4/2025 12:45:00,"$232,562.00"
1,2,3,1/5/2025 14:00:00,"$473,272.00"
2,3,7,1/13/2025 13:01:00,"$280,980.00"


In [ ]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14172 entries, 0 to 14171
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sale_id      14172 non-null  object
 1   customer_id  14172 non-null  object
 2   sale_date    14172 non-null  object
 3   amount       14172 non-null  object
dtypes: object(4)
memory usage: 443.0+ KB


In [ ]:
print("duplicates found",df_sales.duplicated(subset=["sale_id"]).sum())

duplicates found 0


### Calls DataFrame

In [ ]:
df_calls.head(3)

,call_id,customer_id,call_date,disposition,call_handler
0,1,1,1/1/25 0:56,Voicemail,NA
1,2,1,1/4/25 0:21,Voicemail,NA
2,3,2,1/1/25 2:31,Voicemail,NA


In [ ]:
df_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175359 entries, 0 to 175358
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   call_id       175359 non-null  object
 1   customer_id   175359 non-null  object
 2   call_date     175359 non-null  object
 3   disposition   175359 non-null  object
 4   call_handler  175359 non-null  object
dtypes: object(5)
memory usage: 6.7+ MB


In [ ]:
print("Unique values for call_handler column:", df_calls.call_handler.unique())

Unique values for call_handler column: ['NA' 'Vivian' 'Geoffrey' 'Carlton' 'Phil' 'Hillary' 'Will']


In [ ]:
print("Unique values for disposition column:", df_calls.disposition.unique())

Unique values for disposition column: ['Voicemail' 'Talked to Customer' 'Disconnected']


In [ ]:
print("duplicates found",df_calls.duplicated(subset=["call_id"]).sum())

duplicates found 0


### Agents DataFrame

In [ ]:
df_agents.head()

,agent_id,first_name,last_name,market_id,buyer_customers_sent_2024,seller_customers_sent_2024,buyer_closes_2024,seller_closes_2024,customer_reviews_2024,avg_rating_2024
0,hiroshi_thompson_560,Hiroshi,Thompson,cleveland_oh,69,131,18,18,5,4.11
1,ali_nelson_663,Ali,Nelson,phoenix_az,24,226,4,32,1,2.94
2,kwame_singh_576,Kwame,Singh,salt_lake_city_ut,141,73,16,25,6,2.58
3,hao_jackson_527,Hao,Jackson,st_louis_mo,7,272,1,40,6,2
4,camila_mohamed_261,Camila,Mohamed,kansas_city_mo,132,35,14,10,1,2.74


In [ ]:
df_agents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   agent_id                    1000 non-null   object
 1   first_name                  1000 non-null   object
 2   last_name                   1000 non-null   object
 3   market_id                   1000 non-null   object
 4   buyer_customers_sent_2024   1000 non-null   object
 5   seller_customers_sent_2024  1000 non-null   object
 6   buyer_closes_2024           1000 non-null   object
 7   seller_closes_2024          1000 non-null   object
 8   customer_reviews_2024       1000 non-null   object
 9   avg_rating_2024             1000 non-null   object
dtypes: object(10)
memory usage: 78.3+ KB


In [ ]:
print("duplicates found",df_agents.duplicated(subset=["agent_id"]).sum())

duplicates found 0


### Customers DataFrame

In [ ]:
df_customers.head(3)

,customer_id,created_at,first_name,last_name,address,city,state,zip_code,market_id,assigned_agent
0,1,2025-01-01 0:56:00,James,Butt,957 Sunset Ave,Fresno,CA,90272,fresno_ca,yuki_ali_72
1,2,2025-01-01 2:31:00,Josephine,Darakjy,1864 Pine Ct,Phoenix,AZ,85575,phoenix_az,sofia_smith_808
2,3,2025-01-01 3:19:00,Art,Venere,5542 Ridge St,Tampa,FL,32138,tampa_fl,mina_carter_615


In [ ]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     50000 non-null  object
 1   created_at      50000 non-null  object
 2   first_name      50000 non-null  object
 3   last_name       50000 non-null  object
 4   address         50000 non-null  object
 5   city            50000 non-null  object
 6   state           50000 non-null  object
 7   zip_code        50000 non-null  object
 8   market_id       50000 non-null  object
 9   assigned_agent  50000 non-null  object
dtypes: object(10)
memory usage: 3.8+ MB


In [ ]:
print("duplicates found",df_customers.duplicated(subset=["customer_id"]).sum())

duplicates found 0


### Markets DataFrame

In [ ]:
df_markets.head(3)

,market_id,census_name,common_name,population,days_on_market_median,price_new_list_median,price_closed_median,listings_on_market_count,listings_price_increase_count,listings_price_decrease_count,listings_closed_count
0,new_york_ny,"New York-Newark-Jersey City, NY-NJ Metro Area","New York, NY",19756722,58,"$849,000","$785,000",36156,362,7062,13542
1,los_angeles_ca,"Los Angeles-Long Beach-Anaheim, CA Metro Area","Los Angeles, CA",13012469,55,"$979,000","$915,000",18409,330,5260,6606
2,chicago_il,"Chicago-Naperville-Elgin, IL-IN Metro Area","Chicago, IL",9359555,52,"$369,900","$360,000",16751,160,6308,8251


In [ ]:
df_markets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   market_id                      50 non-null     object
 1   census_name                    50 non-null     object
 2   common_name                    50 non-null     object
 3   population                     50 non-null     object
 4   days_on_market_median          50 non-null     object
 5   price_new_list_median          50 non-null     object
 6   price_closed_median            50 non-null     object
 7   listings_on_market_count       50 non-null     object
 8   listings_price_increase_count  50 non-null     object
 9   listings_price_decrease_count  50 non-null     object
 10  listings_closed_count          50 non-null     object
dtypes: object(11)
memory usage: 4.4+ KB


In [ ]:
print("duplicates found",df_markets.duplicated(subset=["market_id"]).sum())

duplicates found 0



## 🔹 Scrub - Data Transformation & Feature Engineering

### Sales DataFrame

In [ ]:
df_sales.head(2)

,sale_id,customer_id,sale_date,amount
0,1,2,1/4/2025 12:45:00,"$232,562.00"
1,2,3,1/5/2025 14:00:00,"$473,272.00"


In [ ]:
#amount to float
df_sales["amount"] = clean_currency(df_sales["amount"])
#Extra columns for analysis
#dates
df_sales["sale_date"] = pd.to_datetime(df_sales["sale_date"])
df_sales["year_month"] = df_sales["sale_date"].dt.to_period("M").dt.to_timestamp()
df_sales["year_month_period"] = df_sales["sale_date"].dt.to_period("M")
df_sales["year_month_period"] = df_sales["year_month_period"].dt.to_timestamp()
df_sales["year"] = df_sales["sale_date"].dt.year
df_sales["month"] = df_sales["sale_date"].dt.month
df_sales["sale_dow"] = df_sales["sale_date"].dt.day_name()
df_sales["sale_mmddyy"] = df_sales["sale_date"].dt.date
#commissions
df_sales["agent_commission"] = df_sales["amount"] * 0.02
df_sales["smiley_revenue"] = df_sales["amount"] * 0.01


In [ ]:
df_sales["amount"].describe().map("{:,.0f}".format).to_frame()

,amount
count,"14,172"
mean,"488,066"
std,"346,356"
min,"223,007"
25%,"405,366"
50%,"485,829"
75%,"565,517"
max,"39,592,800"


In [ ]:
df_sales.head(2)

,sale_id,customer_id,sale_date,amount,year_month,year_month_period,year,month,sale_dow,sale_mmddyy,agent_commission,smiley_revenue
0,1,2,2025-01-04 12:45:00,232562.0,2025-01-01,2025-01-01,2025,1,Saturday,2025-01-04,4651.24,2325.62
1,2,3,2025-01-05 14:00:00,473272.0,2025-01-01,2025-01-01,2025,1,Sunday,2025-01-05,9465.44,4732.72


In [ ]:
#@title Compute outliers
# Compute outliers based on amount
outlier_threshold = df_sales["amount"].quantile(0.99)

# Store outlier rows
outliers_df = df_sales[df_sales["amount"] >= outlier_threshold]

# Summarize by month
outlier_summary = (
    outliers_df["year_month"]
    .value_counts()
    .sort_index()
)


In [ ]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14172 entries, 0 to 14171
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   sale_id            14172 non-null  object        
 1   customer_id        14172 non-null  object        
 2   sale_date          14172 non-null  datetime64[ns]
 3   amount             14172 non-null  float64       
 4   year_month         14172 non-null  datetime64[ns]
 5   year_month_period  14172 non-null  datetime64[ns]
 6   year               14172 non-null  int32         
 7   month              14172 non-null  int32         
 8   sale_dow           14172 non-null  object        
 9   sale_mmddyy        14172 non-null  object        
 10  agent_commission   14172 non-null  float64       
 11  smiley_revenue     14172 non-null  float64       
dtypes: datetime64[ns](3), float64(3), int32(2), object(4)
memory usage: 1.2+ MB


### Calls DataFrame

In [ ]:
df_calls.head(2)

,call_id,customer_id,call_date,disposition,call_handler
0,1,1,1/1/25 0:56,Voicemail,NA
1,2,1,1/4/25 0:21,Voicemail,NA


In [ ]:
#to datetime
df_calls['call_date']=pd.to_datetime(df_calls['call_date'],format="%m/%d/%y %H:%M")
print("calls range:",df_calls.call_date.min(),df_calls.call_date.max())
#To set as category since in exploratory step, I found the call_handler and disposition behave like categories

df_calls["disposition"] = df_calls["disposition"].astype("category")
df_calls["call_handler"] = df_calls["call_handler"].replace('NA',None).astype("category")

calls range: 2025-01-01 00:56:00 2025-11-23 12:24:00


In [ ]:
df_calls.head(2)

,call_id,customer_id,call_date,disposition,call_handler
0,1,1,2025-01-01 00:56:00,Voicemail,NaN
1,2,1,2025-01-04 00:21:00,Voicemail,NaN


In [ ]:
df_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175359 entries, 0 to 175358
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   call_id       175359 non-null  object        
 1   customer_id   175359 non-null  object        
 2   call_date     175359 non-null  datetime64[ns]
 3   disposition   175359 non-null  category      
 4   call_handler  77577 non-null   category      
dtypes: category(2), datetime64[ns](1), object(2)
memory usage: 4.3+ MB


### Agents DataFrame

In [ ]:
df_agents.head(3)

,agent_id,first_name,last_name,market_id,buyer_customers_sent_2024,seller_customers_sent_2024,buyer_closes_2024,seller_closes_2024,customer_reviews_2024,avg_rating_2024
0,hiroshi_thompson_560,Hiroshi,Thompson,cleveland_oh,69,131,18,18,5,4.11
1,ali_nelson_663,Ali,Nelson,phoenix_az,24,226,4,32,1,2.94
2,kwame_singh_576,Kwame,Singh,salt_lake_city_ut,141,73,16,25,6,2.58


In [ ]:
#convert from object to Int64
cols_to_int = [
    'buyer_customers_sent_2024',
    'seller_customers_sent_2024',
    'buyer_closes_2024',
    'seller_closes_2024',
    'customer_reviews_2024',
]

for col in cols_to_int:
    df_agents[col] = pd.to_numeric(df_agents[col]).astype("Int64")


print("**Range for values**")
for col in cols_to_int:
    col_min = df_agents[col].min()
    col_max = df_agents[col].max()
    print(f"{col}: {col_min} → {col_max}")


**Range for values**
buyer_customers_sent_2024: 4 → 296
seller_customers_sent_2024: 3 → 477
buyer_closes_2024: 1 → 30
seller_closes_2024: 1 → 50
customer_reviews_2024: 1 → 15


In [ ]:
df_agents['avg_rating_2024']=df_agents['avg_rating_2024'].replace('',None).astype('float')

In [ ]:
df_agents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   agent_id                    1000 non-null   object 
 1   first_name                  1000 non-null   object 
 2   last_name                   1000 non-null   object 
 3   market_id                   1000 non-null   object 
 4   buyer_customers_sent_2024   1000 non-null   Int64  
 5   seller_customers_sent_2024  1000 non-null   Int64  
 6   buyer_closes_2024           1000 non-null   Int64  
 7   seller_closes_2024          1000 non-null   Int64  
 8   customer_reviews_2024       998 non-null    Int64  
 9   avg_rating_2024             998 non-null    float64
dtypes: Int64(5), float64(1), object(4)
memory usage: 83.1+ KB


In [ ]:
print("Agents with no reviews in 2024")
df_agents[df_agents.avg_rating_2024.isna()]

Agents with no reviews in 2024


,agent_id,first_name,last_name,market_id,buyer_customers_sent_2024,seller_customers_sent_2024,buyer_closes_2024,seller_closes_2024,customer_reviews_2024,avg_rating_2024
877,asha_smith_149,Asha,Smith,raleigh_nc,4,17,1,2,<NA>,NaN
902,thomas_ramirez_900,Thomas,Ramirez,new_york_ny,19,14,2,2,<NA>,NaN


### Customers DataFrame

In [ ]:
df_customers.head(3)

,customer_id,created_at,first_name,last_name,address,city,state,zip_code,market_id,assigned_agent
0,1,2025-01-01 0:56:00,James,Butt,957 Sunset Ave,Fresno,CA,90272,fresno_ca,yuki_ali_72
1,2,2025-01-01 2:31:00,Josephine,Darakjy,1864 Pine Ct,Phoenix,AZ,85575,phoenix_az,sofia_smith_808
2,3,2025-01-01 3:19:00,Art,Venere,5542 Ridge St,Tampa,FL,32138,tampa_fl,mina_carter_615


In [ ]:
df_customers['created_at']=pd.to_datetime(df_customers['created_at'])
print("created_at range:",df_customers.created_at.min(),df_customers.created_at.max())

created_at range: 2025-01-01 00:56:00 2025-11-06 17:56:00


In [ ]:
df_customers["city"] = df_customers["city"].astype("category")
print('Unique cities in customers table:',df_customers.city.unique())


Unique cities in customers table: ['Fresno', 'Phoenix', 'Tampa', 'Louisville', 'Cincinnati', ..., 'Detroit', 'Nashville', 'Charlotte', 'Oklahoma City', 'Boston']
Length: 50
Categories (50, object): ['Atlanta', 'Austin', 'Baltimore', 'Birmingham', ..., 'St. Louis', 'Tampa',
                          'Virginia Beach', 'Washington']


In [ ]:
df_customers["state"] = df_customers["state"].astype("category")
print('Unique state in customers table:',df_customers.state.unique())


Unique state in customers table: ['CA', 'AZ', 'FL', 'KY', 'OH', ..., 'TN', 'UT', 'DC', 'OK', 'MA']
Length: 29
Categories (29, object): ['AL', 'AZ', 'CA', 'CO', ..., 'UT', 'VA', 'WA', 'WI']


In [ ]:
print('Unique zip_code in customers table:',df_customers.zip_code.unique())

Unique zip_code in customers table: ['90272' '85575' '32138' ... '46723' '89331' '27547']


In [ ]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   customer_id     50000 non-null  object        
 1   created_at      50000 non-null  datetime64[ns]
 2   first_name      50000 non-null  object        
 3   last_name       50000 non-null  object        
 4   address         50000 non-null  object        
 5   city            50000 non-null  category      
 6   state           50000 non-null  category      
 7   zip_code        50000 non-null  object        
 8   market_id       50000 non-null  object        
 9   assigned_agent  50000 non-null  object        
dtypes: category(2), datetime64[ns](1), object(7)
memory usage: 3.2+ MB


### Markets DataFrame

In [ ]:
df_markets.head(3)

,market_id,census_name,common_name,population,days_on_market_median,price_new_list_median,price_closed_median,listings_on_market_count,listings_price_increase_count,listings_price_decrease_count,listings_closed_count
0,new_york_ny,"New York-Newark-Jersey City, NY-NJ Metro Area","New York, NY",19756722,58,"$849,000","$785,000",36156,362,7062,13542
1,los_angeles_ca,"Los Angeles-Long Beach-Anaheim, CA Metro Area","Los Angeles, CA",13012469,55,"$979,000","$915,000",18409,330,5260,6606
2,chicago_il,"Chicago-Naperville-Elgin, IL-IN Metro Area","Chicago, IL",9359555,52,"$369,900","$360,000",16751,160,6308,8251


In [ ]:
df_markets["price_new_list_median"] = clean_currency(df_markets["price_new_list_median"])
df_markets["price_closed_median"] = clean_currency(df_markets["price_closed_median"])
print("Range for price_new_list_median:",df_markets.price_new_list_median.min(),df_markets.price_new_list_median.max())
print("Range for price_closed_median:",df_markets.price_closed_median.min(),df_markets.price_closed_median.max())

Range for price_new_list_median: 189046.0 1573032.0
Range for price_closed_median: 205878.0 1632174.0


In [ ]:
#create 2 columns more "city", "state" based on common_name
df_markets[["city", "state"]] = df_markets["common_name"].str.split(",", n=1, expand=True)


In [ ]:
cols_to_int = [
    'population',
    'days_on_market_median',
    'listings_on_market_count',
    'listings_price_increase_count',
    'listings_price_decrease_count',
    'listings_closed_count',
]

for col in cols_to_int:
    df_markets[col] = pd.to_numeric(df_markets[col]).astype("Int64")


print("**Range for values**")
for col in cols_to_int:
    col_min = df_markets[col].min()
    col_max = df_markets[col].max()
    print(f"{col}: {col_min} → {col_max}")

**Range for values**
population: 1154320 → 19756722
days_on_market_median: 12 → 92
listings_on_market_count: 1475 → 47180
listings_price_increase_count: 6 → 1304
listings_price_decrease_count: 490 → 13318
listings_closed_count: 669 → 13542


In [ ]:
df_markets.head(3)

,market_id,census_name,common_name,population,days_on_market_median,price_new_list_median,price_closed_median,listings_on_market_count,listings_price_increase_count,listings_price_decrease_count,listings_closed_count,city,state
0,new_york_ny,"New York-Newark-Jersey City, NY-NJ Metro Area","New York, NY",19756722,58,849000.0,785000.0,36156,362,7062,13542,New York,NY
1,los_angeles_ca,"Los Angeles-Long Beach-Anaheim, CA Metro Area","Los Angeles, CA",13012469,55,979000.0,915000.0,18409,330,5260,6606,Los Angeles,CA
2,chicago_il,"Chicago-Naperville-Elgin, IL-IN Metro Area","Chicago, IL",9359555,52,369900.0,360000.0,16751,160,6308,8251,Chicago,IL


# 🔹 Exploration of Data


## Section: Sales DataFrame

In [ ]:
#@title 🏷️ Distribution of Home Sale Prices (0–99th Percentile)


# Filter out extreme outliers (top 1%)
filtered = df_sales[df_sales["amount"] < df_sales["amount"].quantile(0.99)]

# Percentiles
p25 = filtered["amount"].quantile(0.25)
p50 = filtered["amount"].quantile(0.50)
p75 = filtered["amount"].quantile(0.75)
p95 = filtered["amount"].quantile(0.95)

# Modern blue shades
colors = {
    "25th": "#5DADE2",   # light blue
    "50th": "#2E86C1",   # medium blue
    "75th": "#1B4F72",   # dark blue
    "95th": "#154360"    # deep navy-blue
}

# Create histogram
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=filtered["amount"],
    nbinsx=40,
    marker=dict(color="#85C1E9", line=dict(color="white", width=1)),
    opacity=0.9,
    name="Sale Amounts"
))

# Add percentile lines
fig.add_vline(x=p25, line_dash="dash", line_color=colors["25th"], line_width=3)
fig.add_annotation(x=p25, y=1, yref="paper",
                   text="25th", showarrow=False, font=dict(color=colors["25th"]))

fig.add_vline(x=p50, line_dash="dash", line_color=colors["50th"], line_width=3)
fig.add_annotation(x=p50, y=1, yref="paper",
                   text="Median", showarrow=False, font=dict(color=colors["50th"]))

fig.add_vline(x=p75, line_dash="dash", line_color=colors["75th"], line_width=3)
fig.add_annotation(x=p75, y=1, yref="paper",
                   text="75th", showarrow=False, font=dict(color=colors["75th"]))

fig.add_vline(x=p95, line_dash="dash", line_color=colors["95th"], line_width=3)
fig.add_annotation(x=p95, y=1, yref="paper",
                   text="95th", showarrow=False, font=dict(color=colors["95th"]))

# Layout
fig.update_layout(
    title="<b>🏷️ Distribution of Home Sale Prices (0–99th Percentile)</b>",
    title_font_size=22,
    bargap=0.05,
    plot_bgcolor="white",
    width=1000,
    height=550,
    xaxis_title="🏷️ Sale Price ($)",
    yaxis_title="Frequency",
)

# Y-axis styling
fig.update_yaxes(showgrid=True, gridcolor="#EBF5FB")
fig.update_xaxes(showgrid=False)

# Format currency
fig.update_xaxes(tickprefix="$")
fig.show()

print(f"25th Percentile (p25 = ${p25:,.0f}):")
print("  25% of home sale prices fall below this value.\n"
      "  These are typically lower-priced or entry-level homes.\n")

print(f"Median (p50 = ${p50:,.0f}):")
print("  Half of all home sales fall below the median and half above.\n"
      "  This represents the typical home price in the dataset.\n")

print(f"75th Percentile (p75 = ${p75:,.0f}):")
print("  75% of home sales fall below this value.\n"
      "  Homes above this mark are in the higher-priced segment.\n")

print(f"95th Percentile (p95 = ${p95:,.0f}):")
print("  95% of home sale prices fall below this value.\n"
      "  Properties above this point represent the most expensive 5% of the market.\n")
print_outlier_message()


25th Percentile (p25 = $404,313):
  25% of home sale prices fall below this value.
  These are typically lower-priced or entry-level homes.

Median (p50 = $484,184):
  Half of all home sales fall below the median and half above.
  This represents the typical home price in the dataset.

75th Percentile (p75 = $563,108):
  75% of home sales fall below this value.
  Homes above this mark are in the higher-priced segment.

95th Percentile (p95 = $658,989):
  95% of home sale prices fall below this value.
  Properties above this point represent the most expensive 5% of the market.


🔍 Outlier threshold (99th percentile): $713,178

⚠️ Outliers removed from the following month-year groups:

  • Mar 2025: 1 sale(s)
  • Oct 2025: 74 sale(s)
  • Nov 2025: 67 sale(s)




In [ ]:
#@title 🏷️ Distribution of Home Sale Prices per Month (0–99th Percentile)
filtered = df_sales[df_sales["amount"] < df_sales["amount"].quantile(0.99)]

fig = px.box(
    filtered,
    x="year_month",
    y="amount",
    title="🏷️ Distribution of Home Sale Prices per Month (0–99th Percentile)",
    labels={"year_month": "Month-Year", "amount": "Sale Price ($)"},
    color="year_month",
    points=False
)

fig.update_layout(
    xaxis_tickangle=-45,
    plot_bgcolor="white",
    showlegend=False
)

fig.update_yaxes(tickprefix="$")
fig.show()

print_outlier_message()




🔍 Outlier threshold (99th percentile): $713,178

⚠️ Outliers removed from the following month-year groups:

  • Mar 2025: 1 sale(s)
  • Oct 2025: 74 sale(s)
  • Nov 2025: 67 sale(s)




In [ ]:
#@title 🏷️ Median Home $ Sale Price per Month

# 1️⃣ Median per month
median_monthly_sales = (
    df_sales.groupby("year_month_period")["amount"]
    .median()
    .to_frame("median_amount")
)

# 2️⃣ Convert Period → Timestamp
median_monthly_sales.index = median_monthly_sales.index

# 3️⃣ Add percent difference vs previous month
median_monthly_sales["pct_change"] = median_monthly_sales["median_amount"].pct_change() * 100

# 4️⃣ Reset for Plotly
median_monthly_sales.reset_index(inplace=True)

# 5️⃣ Build chart
fig = px.bar(
    median_monthly_sales,
    x="year_month_period",
    y="median_amount",
    title="🏷️ Median Home Sale Price per Month",
    labels={
        "year_month_period": "Month-Year",
        "median_amount": "Median Sale Price ($)"
    },
    color="median_amount",
    color_continuous_scale="Blues",
    text_auto=".2s"
)

# 6️⃣ Pretty layout
fig.update_layout(
    title_font_size=22,
    xaxis_tickangle=-45,
    plot_bgcolor="white",
    bargap=0.2
)

# 7️⃣ Currency formatting
fig.update_yaxes(tickprefix="$", gridcolor="lightgray")

# 8️⃣ Add % change into hover label
fig.update_traces(
    hovertemplate="<b>%{x|%b %Y}</b><br>"
                  "🏷️ Median Price: %{y:$,.0f}<br>"
                  "MoM Change: %{customdata:.1f}%<extra></extra>",
    customdata=median_monthly_sales["pct_change"]
)

fig.show()
avg_pct_change = median_monthly_sales["pct_change"].mean()
median_pct_change = median_monthly_sales["pct_change"].median()
print(f"Median monthly % change in median sale price: {median_pct_change:.2f}%")
print(f"Average monthly % change in median sale price: {avg_pct_change:.2f}%")


Median monthly % change in median sale price: 4.40%
Average monthly % change in median sale price: 5.27%


In [ ]:
#@title 🏡 Monthly Count of Sales

import plotly.graph_objects as go

# --- Monthly count of sales ---
monthly_count = (
    df_sales.groupby("year_month")["sale_id"]
    .count()
    .sort_index()
)

# --- Month-over-month % change ---
monthly_count_pct = monthly_count.pct_change() * 100
avg_pct_change = monthly_count_pct.mean()
median_pct_change = monthly_count_pct.median()
avg_abs_pct_change = monthly_count_pct.abs().mean()

# --- Rolling 3-month trend ---
monthly_count_rolling = monthly_count.rolling(3).mean()

# --- Max & Min ---
max_month = monthly_count.idxmax()
max_value = monthly_count.max()

min_month = monthly_count.idxmin()
min_value = monthly_count.min()

# --- Build chart ---
fig = go.Figure()

# 1️⃣ Main purple line
fig.add_trace(go.Scatter(
    x=monthly_count.index,
    y=monthly_count.values,
    mode="lines+markers",
    name="Monthly Sales Count",
    line=dict(color="#6A0DAD", width=4),
    marker=dict(size=8, color="#6A0DAD")
))

# 2️⃣ Soft purple trend line
fig.add_trace(go.Scatter(
    x=monthly_count.index,
    y=monthly_count_rolling.values,
    mode="lines",
    name="3-Month Trend",
    line=dict(color="#9B5DE5", width=3)
))

# 3️⃣ Highlight max (NO HOVER)
fig.add_trace(go.Scatter(
    x=[max_month],
    y=[max_value],
    mode="markers+text",
    name="Highest Month",
    marker=dict(color="#5A189A", size=12),
    text=[f"<b>{max_value:,} 🏡</b>"],
    textposition="top center",
    hoverinfo="skip",
    hovertemplate=None,
    showlegend=False
))

# 4️⃣ Highlight min (NO HOVER)
fig.add_trace(go.Scatter(
    x=[min_month],
    y=[min_value],
    mode="markers+text",
    name="Lowest Month",
    marker=dict(color="#B5179E", size=12),
    text=[f"<b>{min_value:,} 🏡</b>"],
    textposition="bottom center",
    hoverinfo="skip",
    hovertemplate=None,
    showlegend=False
))

# --- Apply hover ONLY to main two traces ---
fig.data[0].update(
    hovertemplate="<b>%{x|%b %Y}</b><br>"
                  "🏡 Sales Count: %{y:,}<br>"
                  "MoM Change: %{customdata:.1f}%<extra></extra>",
    customdata=monthly_count_pct
)

fig.data[1].update(
    hovertemplate="<b>%{x|%b %Y}</b><br>"
                  "Trend Value: %{y:,} 🏡<br>"
                  "MoM Change: %{customdata:.1f}%<extra></extra>",
    customdata=monthly_count_pct
)

# --- Layout ---
fig.update_layout(
    title="<b>Monthly Count of Sales</b>",
    width=1000,
    height=550,
    plot_bgcolor="white",
    xaxis_title="Month",
    yaxis_title="Number of Sales",
    font=dict(family="Arial", size=12, color="#333"),
)

fig.update_yaxes(showgrid=True, gridcolor="#EAEAEA")
fig.update_xaxes(showgrid=False, tickangle=-45)

fig.show()

# --- Summary ---
print(f"Average MoM % change: {avg_pct_change:.2f}%")
print(f"Median MoM % change: {median_pct_change:.2f}%")
print(f"Average Monthly Volatility (Abs Change): {avg_abs_pct_change:.2f}%")
print(f"📉 Lowest month: {min_month.strftime('%b %Y')} — {min_value:,} sales 🏡")
print(f"📈 Highest month: {max_month.strftime('%b %Y')} — {max_value:,} sales 🏡")


Average MoM % change: -2.57%
Median MoM % change: 3.38%
Average Monthly Volatility (Abs Change): 12.74%
📉 Lowest month: Nov 2025 — 574 sales 🏡
📈 Highest month: Jul 2025 — 1,530 sales 🏡


In [ ]:
#@title 💰 Monthly Home Sales $ Trend

import plotly.graph_objects as go

# --- Monthly total sales (raw, with outliers) ---
monthly_sales = (
    df_sales.groupby("year_month")["amount"]
    .sum()
    .sort_index()
)

# --- Month-over-month % change ---
monthly_pct_change = monthly_sales.pct_change() * 100
avg_pct_change = monthly_pct_change.mean()
median_pct_change = monthly_pct_change.median()
avg_abs_pct_change = monthly_pct_change.abs().mean()

# --- Rolling 3-month trend ---
monthly_sales_rolling = monthly_sales.rolling(3).mean()

# --- Max & Min month ---
max_month = monthly_sales.idxmax()
max_value = monthly_sales.max()

min_month = monthly_sales.idxmin()
min_value = monthly_sales.min()

# --- Build chart ---
fig = go.Figure()

# 1️⃣ Monthly sales line
fig.add_trace(go.Scatter(
    x=monthly_sales.index,
    y=monthly_sales.values,
    mode="lines+markers",
    name="Monthly Sales",
    line=dict(color="#0077CC", width=4),
    marker=dict(size=8, color="#0077CC")
))

# 2️⃣ Rolling trend line
fig.add_trace(go.Scatter(
    x=monthly_sales.index,
    y=monthly_sales_rolling.values,
    mode="lines",
    name="3-Month Trend",
    line=dict(color="#00A6FB", width=3)
))

# 3️⃣ Highlight max (NO HOVER)
fig.add_trace(go.Scatter(
    x=[max_month],
    y=[max_value],
    mode="markers+text",
    name="Highest Month",
    marker=dict(color="#3A0CA3", size=12),
    text=[f"<b>${max_value:,.0f}</b>"],
    textposition="top center",
    hoverinfo="skip",
    hovertemplate=None,
    showlegend=False
))

# 4️⃣ Highlight min (NO HOVER)
fig.add_trace(go.Scatter(
    x=[min_month],
    y=[min_value],
    mode="markers+text",
    name="Lowest Month",
    marker=dict(color="#8E44AD", size=12),
    text=[f"<b>${min_value:,.0f}</b>"],
    textposition="bottom center",
    hoverinfo="skip",
    hovertemplate=None,
    showlegend=False
))

# --- Apply hover ONLY to the first two traces ---
fig.data[0].update(
    hovertemplate="<b>%{x|%b %Y}</b><br>"
                  "💰 Sales: %{y:$,.0f}<br>"
                  "MoM Change: %{customdata:.1f}%<extra></extra>",
    customdata=monthly_pct_change
)

fig.data[1].update(
    hovertemplate="<b>%{x|%b %Y}</b><br>"
                  "Trend Value: %{y:$,.0f}<br>"
                  "MoM Change: %{customdata:.1f}%<extra></extra>",
    customdata=monthly_pct_change
)

# --- Layout ---
fig.update_layout(
    title="<b>💰 Monthly Home Sales $ Trend (With Outliers)</b>",
    width=1000,
    height=550,
    plot_bgcolor="white",
    xaxis_title="Month",
    yaxis_title="Total Sales ($)"
)

fig.update_yaxes(showgrid=True, gridcolor="#E5F2FA", tickprefix="$")
fig.update_xaxes(showgrid=False, tickangle=-45)

fig.show()

# --- Summary below chart ---
print(f"Average MoM % change: {avg_pct_change:.2f}%")
print(f"Median MoM % change: {median_pct_change:.2f}%")
print(f"Average Monthly Volatility (Absolute Change): {avg_abs_pct_change:.2f}%")
print(f"📉 Lowest month: {min_month.strftime('%b %Y')} — ${min_value:,.0f}")
print(f"📈 Highest month: {max_month.strftime('%b %Y')} — ${max_value:,.0f}")


Average MoM % change: 2.64%
Median MoM % change: 8.35%
Average Monthly Volatility (Absolute Change): 17.10%
📉 Lowest month: Nov 2025 — $345,105,691
📈 Highest month: Oct 2025 — $851,586,798


In [ ]:
#@title 💰 Monthly Home Sales $ Trend (Outliers Removed)

# --- Remove outliers (top 1%) ---
threshold = df_sales["amount"].quantile(0.99)
df_no_outliers = df_sales[df_sales["amount"] < threshold]

# --- Monthly total sales (cleaned) ---
monthly_sales = (
    df_no_outliers.groupby("year_month")["amount"]
    .sum()
    .sort_index()
)

# --- Month-over-month % change ---
monthly_pct_change = monthly_sales.pct_change() * 100
avg_pct_change = monthly_pct_change.mean()
median_pct_change = monthly_pct_change.median()
avg_abs_pct_change = monthly_pct_change.abs().mean()

# --- Rolling 3-month trend ---
monthly_sales_rolling = monthly_sales.rolling(3).mean()

# --- Max & Min month ---
max_month = monthly_sales.idxmax()
max_value = monthly_sales.max()

min_month = monthly_sales.idxmin()
min_value = monthly_sales.min()

# --- Build chart ---
fig = go.Figure()

# 1️⃣ Main sales line
fig.add_trace(go.Scatter(
    x=monthly_sales.index,
    y=monthly_sales.values,
    mode="lines+markers",
    name="Monthly Sales (No Outliers)",
    line=dict(color="#0077CC", width=4),
    marker=dict(size=8, color="#0077CC")
))

# 2️⃣ Trend line
fig.add_trace(go.Scatter(
    x=monthly_sales.index,
    y=monthly_sales_rolling.values,
    mode="lines",
    name="3-Month Trend",
    line=dict(color="#00A6FB", width=3)
))

# 3️⃣ Highlight max (NO HOVER)
fig.add_trace(go.Scatter(
    x=[max_month],
    y=[max_value],
    mode="markers+text",
    name="Highest Month",
    marker=dict(color="#3A0CA3", size=12),
    text=[f"<b>${max_value:,.0f}</b>"],
    textposition="top center",
    hoverinfo="skip",
    hovertemplate="",
    showlegend=False
))

# 4️⃣ Highlight min (NO HOVER)
fig.add_trace(go.Scatter(
    x=[min_month],
    y=[min_value],
    mode="markers+text",
    name="Lowest Month",
    marker=dict(color="#8E44AD", size=12),
    text=[f"<b>${min_value:,.0f}</b>"],
    textposition="bottom center",
    hoverinfo="skip",
    hovertemplate="",
    showlegend=False
))

# --- Hover ONLY for traces 0 and 1 ---
fig.data[0].update(
    hovertemplate="<b>%{x|%b %Y}</b><br>"
                  "💰Sales: %{y:$,.0f}<br>"
                  "MoM Change: %{customdata:.1f}%<extra></extra>",
    customdata=monthly_pct_change
)

fig.data[1].update(
    hovertemplate="<b>%{x|%b %Y}</b><br>"
                  "Trend Value: %{y:$,.0f}<br>"
                  "MoM Change: %{customdata:.1f}%<extra></extra>",
    customdata=monthly_pct_change
)

# --- Layout ---
fig.update_layout(
    title="<b>💰 Monthly Home Sales $ Trend (Outliers Removed)</b>",
    width=1000, height=550,
    plot_bgcolor="white",
    xaxis_title="Month",
    yaxis_title="Total Sales ($)"
)

fig.update_yaxes(showgrid=True, gridcolor="#E5F2FA", tickprefix="$")
fig.update_xaxes(showgrid=False, tickangle=-45)

fig.show()

# --- Summary ---
print(f"Average MoM % change: {avg_pct_change:.2f}%")
print(f"Median MoM % change: {median_pct_change:.2f}%")
print(f"Average Monthly Volatility (Absolute Change): {avg_abs_pct_change:.2f}%")
print(f"📉 Lowest month (no outliers): {min_month.strftime('%b %Y')} — ${min_value:,.0f}")
print(f"📈 Highest month (no outliers): {max_month.strftime('%b %Y')} — ${max_value:,.0f}")
print_outlier_message()


Average MoM % change: 1.40%
Median MoM % change: 5.65%
Average Monthly Volatility (Absolute Change): 15.41%
📉 Lowest month (no outliers): Nov 2025 — $296,135,510
📈 Highest month (no outliers): Oct 2025 — $798,246,768

🔍 Outlier threshold (99th percentile): $713,178

⚠️ Outliers removed from the following month-year groups:

  • Mar 2025: 1 sale(s)
  • Oct 2025: 74 sale(s)
  • Nov 2025: 67 sale(s)




# 🔹 Interpret Stage
Charts created for interpretation step, interpretation is available at slides

In [ ]:
#@title Join Tables
query1 = """
SELECT
    s.sale_id,
    s.amount,
    s.customer_id,
    a.agent_id,
    m.common_name as market_name,
    c.state,
    a.first_name ||' '|| a.last_name AS agent_name,
    c.first_name ||' '|| c.last_name AS customer_name,
    s.agent_commission,
    s.smiley_revenue
FROM df_sales s
LEFT JOIN df_customers c
    ON s.customer_id = c.customer_id
LEFT JOIN df_agents a
    ON c.assigned_agent = a.agent_id
LEFT JOIN df_markets m
    ON a.market_id = m.market_id
"""

df_sales_by_agent = duckdb.query(query1).df()

df_sales_by_agent.head()

,sale_id,amount,customer_id,agent_id,market_name,state,agent_name,customer_name,agent_commission,smiley_revenue
0,1,232562.0,2,sofia_smith_808,"Phoenix, AZ",AZ,Sofia Smith,Josephine Darakjy,4651.24,2325.62
1,2,473272.0,3,mina_carter_615,"Tampa, FL",FL,Mina Carter,Art Venere,9465.44,4732.72
2,3,280980.0,7,arjun_ramirez_306,"Cleveland, OH",OH,Arjun Ramirez,Mitsue Tollner,5619.60,2809.80
3,4,500904.0,15,ali_hall_3,"Cleveland, OH",OH,Ali Hall,Cammy Albares,10018.08,5009.04
4,5,247848.0,17,arjun_ramirez_306,"Cleveland, OH",OH,Arjun Ramirez,Meaghan Garufi,4956.96,2478.48


## Section: Agents
 - This analysis uses agent_id as the unique identifier for agents.
 - The dataset contains homonymous agents (agents with the same first and last name), and the table specification explicitly states that agent_id is the unique key.

In [ ]:
# @title 👩‍💼 Top 10 Agents — Revenue, Commission, Sales Count, Median Sale (Dropdown)

import plotly.graph_objects as go
import duckdb
import pandas as pd

# --- 1. Build the summary dataframe ---
query = """
SELECT
    agent_id,
    agent_name,
    state,
    SUM(smiley_revenue) AS total_revenue,
    SUM(agent_commission) AS total_commission,
    SUM(amount) AS total_amount,
    COUNT(*) AS sales_count,
    MEDIAN(amount) AS median_sale
FROM df_sales_by_agent
GROUP BY agent_id, agent_name, state
"""

df = duckdb.query(query).df()

# --- 2. Define metrics for dropdown ---
metrics = {
    "Total Revenue (Smiley)": "total_revenue",
    "Agent Commission": "total_commission",
    "Total Sale Amount": "total_amount",
    "Sales Count": "sales_count",
    "Median Sale Amount": "median_sale",
}

# Sort dynamically for the FIRST metric (default view)
default_metric_label = "Total Revenue (Smiley)"
default_metric = metrics[default_metric_label]

df_sorted = df.sort_values(default_metric, ascending=False).head(10)
df_sorted = df_sorted.iloc[::-1]  # reverse for horizontal bar ASC

# --- 3. Customdata for hover ---
cd = df_sorted[[
    "state",
    "total_revenue",
    "total_commission",
    "sales_count",
    "median_sale",
    "agent_id"
]].values

# --- 4. Build the figure ---
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_sorted[default_metric],
    y=df_sorted["agent_name"],
    orientation="h",
    customdata=cd,
    hovertemplate=(
        "<b>%{y}</b><br>" +
        "Agent ID: %{customdata[5]}<br>" +
        "State: %{customdata[0]}<br>" +
        "Revenue (Smiley): %{customdata[1]:,.0f}<br>" +
        "Commission (Agent): %{customdata[2]:,.0f}<br>" +
        "Sales Count: %{customdata[3]}<br>" +
        "Median Sale: %{customdata[4]:,.0f}<extra></extra>"
    )
))

# --- 5. Dropdown buttons ---
buttons = []
for label, col in metrics.items():
    buttons.append(dict(
        label=label,
        method="update",
        args=[
            {
                "x": [df.sort_values(col, ascending=False).head(10).iloc[::-1][col]],
                "y": [df.sort_values(col, ascending=False).head(10).iloc[::-1]["agent_name"]],
                "customdata": [
                    df.sort_values(col, ascending=False)
                    .head(10).iloc[::-1][[
                        "state",
       			        "total_revenue",
                        "total_commission",
                        "sales_count",
                        "median_sale",
                        "agent_id"
                    ]].values
                ],
            },
            {
                "title": f"Top 10 Agents — {label}"
            }
        ]
    ))

fig.update_layout(
    title=f"Top 10 Agents — {default_metric_label}",
    xaxis_title="Value",
    yaxis=dict(automargin=True),
    height=600,
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=0.05,
        y=1.09
    )]
)

fig.show()
print_agent_message()

This analysis uses agent_id as the unique identifier for agents.The dataset contains homonymous agents (agents with the same first and last name), and the table specification explicitly states that agent_id is the unique key.


## Section: Markets

In [ ]:
#@title 🏷️  Distribution of Home Sale Prices by Market (0–99th Percentile)


# ---------------------------------------------------------
# 1. Compute 99th percentile threshold
# ---------------------------------------------------------
q99 = df_sales_by_agent["amount"].quantile(0.99)

# Identify outliers
df_outliers = df_sales_by_agent[df_sales_by_agent["amount"] > q99]

# Filtered dataset
df_filtered = df_sales_by_agent[df_sales_by_agent["amount"] <= q99]

# ---------------------------------------------------------
# 2. Print summary
# ---------------------------------------------------------
print(f"Total outliers removed: {len(df_outliers)}")
print(f"99th percentile threshold: ${q99:,.0f}")

# ---------------------------------------------------------
# 3. Boxplot by market (cleaned data)
# ---------------------------------------------------------
fig = px.box(
    df_filtered,
    x="market_name",
    y="amount",
    points="outliers",
    title="Distribution of Home Sale Prices by Market (0–99th Percentile)",
)

fig.update_layout(
    xaxis_tickangle=45,
    height=650,
    template="plotly_white",
    xaxis_title="Market",
    yaxis_title="Sale Amount ($)"
)

fig.show()


Total outliers removed: 142
99th percentile threshold: $713,178


In [ ]:
#@title 🏷️ Distribution of Home Sale Prices by State (0–99th Percentile)

# ---------------------------------------------------------
# 1. Compute 99th percentile threshold
# ---------------------------------------------------------
q99 = df_sales_by_agent["amount"].quantile(0.99)

# Identify outliers
df_outliers = df_sales_by_agent[df_sales_by_agent["amount"] > q99]

# Filtered dataset
df_filtered = df_sales_by_agent[df_sales_by_agent["amount"] <= q99]

# ---------------------------------------------------------
# 2. Print summary
# ---------------------------------------------------------
print(f"Total outliers removed: {len(df_outliers)}")
print(f"99th percentile threshold: ${q99:,.0f}")

# ---------------------------------------------------------
# 3. Boxplot by STATE (cleaned)
# ---------------------------------------------------------
fig = px.box(
    df_filtered,
    x="state",
    y="amount",
    points="outliers",
    title="Distribution of Home Sale Prices by State (0–99th Percentile)",
)

fig.update_layout(
    xaxis_tickangle=45,
    height=650,
    template="plotly_white",
    xaxis_title="State",
    yaxis_title="Sale Amount ($)"
)

fig.show()


Total outliers removed: 142
99th percentile threshold: $713,178


In [ ]:
#@title 🔒 Distribution of Closed Sale Prices by Market

# Sort markets by median closed price
df_sorted = df_markets.sort_values("price_closed_median", ascending=False)

fig = px.bar(
    df_sorted,
    x="common_name",
    y="price_closed_median",
    title="🔒 Median Closed Sale Price by Market (Sorted Highest → Lowest)",
    text_auto=".2s",  # shows values on top of bars
)

fig.update_layout(
    xaxis_title="Market",
    yaxis_title="Median Closed Price",
    height=650,
    xaxis_tickangle=45,
    bargap=0.2
)

fig.show()



In [ ]:
#@title 🔒 Distribution of Closed Median Prices Across Markets
px.box(df_markets, y="price_closed_median", title="Distribution of Closed Median Prices Across Markets")


## 📊 Spearman Correlation Matrix for Key Market Variables


In [ ]:
# @title Spearman Correlation Matrix for Key Market Variables

# -------------------------
# 1. Variables to include
# -------------------------
corr_vars = [
    "population",
    "price_closed_median",
    "days_on_market_median",
    "listings_closed_count",
    "listings_price_decrease_count",
]

# -------------------------
# 2. Compute Spearman correlation
# -------------------------
corr = df_markets[corr_vars].corr(method="spearman")

# -------------------------
# 3. Plotly Correlation Heatmap
# -------------------------
fig = px.imshow(
    corr,
    text_auto=".2f",
    color_continuous_scale="Blues",
    title="📊 Spearman Correlation Matrix for Key Market Variables",
    aspect="auto"
)

fig.update_layout(
    width=750,
    height=650,
    xaxis_title="Variables",
    yaxis_title="Variables",
    coloraxis_colorbar=dict(title="Correlation"),
)

fig.show()

# -------------------------------------------------
# 4. Automated Interpretation Function (Very Clean)
# -------------------------------------------------
def interpret_correlation(value):
    """Return strength + direction text for the correlation."""
    direction = "positive" if value > 0 else "negative" if value < 0 else "no"

    strength = (
        "very weak" if abs(value) < 0.2 else
        "weak" if abs(value) < 0.4 else
        "moderate" if abs(value) < 0.6 else
        "strong"
    )

    return f"{strength} {direction} correlation"

# -------------------------------------------------
# 5. Generate analysis ONLY for pairs with |ρ| ≥ 0.5
# -------------------------------------------------
print("📝 ANALYSIS OF STRONG CORRELATIONS (|ρ| ≥ 0.5)\n")

threshold = 0.5
already_done = set()

for row in corr.index:
    for col in corr.columns:
        if row == col:
            continue

        pair = tuple(sorted([row, col]))
        if pair in already_done:
            continue

        value = corr.loc[row, col]

        # Only print correlations >= 0.5 or <= -0.5
        if abs(value) >= threshold:
            interpretation = interpret_correlation(value)
            print(f"🔹 {row} vs {col}: {value:.3f} → {interpretation}")
            print("   → Strong relationship — likely meaningful for forecasting or strategy.\n")

        already_done.add(pair)


📝 ANALYSIS OF STRONG CORRELATIONS (|ρ| ≥ 0.5)

🔹 population vs listings_closed_count: 0.938 → strong positive correlation
   → Strong relationship — likely meaningful for forecasting or strategy.

🔹 population vs listings_price_decrease_count: 0.791 → strong positive correlation
   → Strong relationship — likely meaningful for forecasting or strategy.

🔹 days_on_market_median vs listings_price_decrease_count: 0.625 → strong positive correlation
   → Strong relationship — likely meaningful for forecasting or strategy.

🔹 listings_closed_count vs listings_price_decrease_count: 0.811 → strong positive correlation
   → Strong relationship — likely meaningful for forecasting or strategy.



#### **Analysis of Strong Spearman Correlations (|ρ| ≥ 0.5)**

Some strong correlations, like population with listings_closed_count (ρ = 0.938) and closed listings with price decreases (ρ = 0.811), are mostly driven by market size. Larger markets naturally have more activity, so these patterns offer limited strategic insight.

The more meaningful findings are those not explained by scale. **Population’s correlation with listings_price_decrease_count (ρ = 0.791)** suggests more competitive pricing dynamics in bigger markets, where sellers adjust prices more often. Likewise, **days_on_market_median correlates with listings_price_decrease_count (ρ = 0.625)**, indicating that slower-moving listings tend to receive more price reductions as sellers respond to weaker demand.

Overall, the key insights center on **pricing competition in larger markets** and **price cuts triggered by longer time on market**.


In [ ]:
#@title Market Position: Population vs Price Decrease count (Competitive pricing dynamics)
px.scatter(
    df_markets,
    x="population",
    y="listings_price_decrease_count",
    color="state",
    hover_name="common_name",
    title="Market Position: Population vs Price Decrease count"
)

In [ ]:
#@title Market Position: Days on Market vs Price Decrease count (adjustments in response to slower demand)
px.scatter(
    df_markets,
    x="days_on_market_median",
    y="listings_price_decrease_count",
    size="population",
    color="state",
    hover_name="common_name",
    title="Market Position: Days on Market vs Price Decrease count"
)


In [ ]:
#@title 📊 Top 10 Markets — Select Metric (Closed vs Price Decreases)

import plotly.graph_objects as go
import numpy as np

# --- PREPARE SORTED TOP 10 DATASETS ---
df_closed = (
    df_markets
    .sort_values("listings_closed_count", ascending=False)
    .head(10)
)

df_decreases = (
    df_markets
    .sort_values("listings_price_decrease_count", ascending=False)
    .head(10)
)

# Extract values
markets_closed = df_closed["common_name"].tolist()[::-1]
closed_values = df_closed["listings_closed_count"].tolist()[::-1]

markets_decreases = df_decreases["common_name"].tolist()[::-1]
decrease_values = df_decreases["listings_price_decrease_count"].tolist()[::-1]

# --- INITIAL VIEW (Closed Listings) ---
fig = go.Figure()

fig.add_trace(go.Bar(
    x=closed_values,
    y=markets_closed,
    orientation="h",
    name="Listings Closed",
    text=[f"{v:,}" for v in closed_values],
    textposition="outside"
))

# --- DROPDOWN ---
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=[
                dict(
                    label="Listings Closed",
                    method="update",
                    args=[
                        {
                            "x": [closed_values],
                            "y": [markets_closed],
                            "text": [[f"{v:,}" for v in closed_values]]
                        },
                        {"title": "📈 Top 10 Markets — Listings Sold/Closed (Last 30 Days)"}
                    ]
                ),
                dict(
                    label="Price Decreases",
                    method="update",
                    args=[
                        {
                            "x": [decrease_values],
                            "y": [markets_decreases],
                            "text": [[f"{v:,}" for v in decrease_values]]
                        },
                        {"title": "📉 Top 10 Markets — Listings with Price Decreases (Last 30 Days)"}
                    ]
                ),
            ],
            x=0.5,
            xanchor="center",
            y=1.09,
            yanchor="top",
            direction="down",
            showactive=True
        )
    ]
)

# --- STYLING ---
fig.update_layout(
    title="📈 Top 10 Markets — Listings Sold/Closed (Last 30 Days)",
    xaxis_title="Count",
    yaxis_title="Market",
    height=600,
    margin=dict(l=150)
)

fig.show()


In [ ]:
# @title 📊 Top 10 Markets — Choose a Metric (Sales + Agent Commission + Smiley Revenue)

import plotly.graph_objects as go

# --------------------------
# 1. Build Market Aggregates
# --------------------------
df_market = (
    df_sales_by_agent.groupby("market_name", as_index=False)
    .agg(
        total_sales=("amount", "sum"),
        avg_sale=("amount", "mean"),
        median_sale=("amount", "median"),
        num_sales=("sale_id", "nunique"),
        agent_commission_sum=("agent_commission", "sum"),
        agent_commission_median=("agent_commission", "median"),
        smiley_revenue_sum=("smiley_revenue", "sum"),
        smiley_revenue_median=("smiley_revenue", "median"),
    )
)

# -----------------------------
# 2. TOP 10 LISTS FOR EACH METRIC
# -----------------------------
metrics = {
    "Total Sales": ("total_sales", "🏆 Top 10 Markets by Total Sales"),
    "Average Sale": ("avg_sale", "📈 Top 10 Markets by Average Sale"),
    "Median Sale": ("median_sale", "📐 Top 10 Markets by Median Sale"),
    "# of Sales": ("num_sales", "🔢 Top 10 Markets by # of Sales"),
    "Agent Commission (Sum)": ("agent_commission_sum", "💼 Top 10 Markets by Agent Commission (Sum)"),
    "Agent Commission (Median)": ("agent_commission_median", "💼 Top 10 Markets by Agent Commission (Median)"),
    "Smiley Revenue (Sum)": ("smiley_revenue_sum", "💰 Top 10 Markets by Smiley Revenue (Sum)"),
    "Smiley Revenue (Median)": ("smiley_revenue_median", "💰 Top 10 Markets by Smiley Revenue (Median)"),
}

# Prepare sorted lists & order for y-axis alignment
top10_data = {}
orders = {}

for label, (col, _) in metrics.items():
    top10 = df_market.nlargest(10, col)
    order = top10.sort_values(col, ascending=False)["market_name"].tolist()[::-1]  # reversed for horizontal bars
    top10_data[label] = top10.set_index("market_name")[col]
    orders[label] = order

# --------------------------
# 3. Create Figure with 8 Traces
# --------------------------
fig = go.Figure()

visibility_mask = []
for i, (label, series) in enumerate(top10_data.items()):
    fig.add_trace(
        go.Bar(
            x=series.loc[orders[label]],
            y=orders[label],
            orientation="h",
            name=label,
            visible=(i == 0),  # show only first metric initially
        )
    )
    visibility_mask.append(i == 0)

# --------------------------
# 4. Dropdown Buttons
# --------------------------
buttons = []
for i, (label, (_, title)) in enumerate(metrics.items()):
    visibility = [False] * len(metrics)
    visibility[i] = True

    buttons.append(
        dict(
            label=label,
            method="update",
            args=[
                {"visible": visibility},
                {
                    "title": title,
                    "yaxis": {
                        "categoryorder": "array",
                        "categoryarray": orders[label]
                    }
                }
            ]
        )
    )

# --------------------------
# 5. Final Layout
# --------------------------
fig.update_layout(
    title=list(metrics.values())[0][1],
    height=650,
    xaxis_title="Metric Value",
    yaxis_title="Market",
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            showactive=True,
            x=0.5,
            xanchor="center",
            y=1.15,
            yanchor="top",
        )
    ]
)

fig.show()


In [ ]:
# @title 📊 Top 10 Markets — Choose a Metric (Sales + Commission + Smiley Revenue + Per-Agent KPIs)

import plotly.graph_objects as go
import pandas as pd

# --------------------------
# 1. Build Market Aggregates (CORRECTED)
# --------------------------
# First compute agent_count per market
agent_counts = (
    df_sales_by_agent.groupby("market_name")["agent_id"]
    .nunique()
    .rename("agent_count")
)

# Now compute all metrics
df_market = (
    df_sales_by_agent.groupby("market_name", as_index=False)
    .agg(
        total_sales=("amount", "sum"),
        avg_sale=("amount", "mean"),
        median_sale=("amount", "median"),
        num_sales=("sale_id", "nunique"),

        # Agent commission totals & medians
        agent_commission_sum=("agent_commission", "sum"),
        agent_commission_median=("agent_commission", "median"),

        # Smiley revenue totals & medians
        smiley_revenue_sum=("smiley_revenue", "sum"),
        smiley_revenue_median=("smiley_revenue", "median"),
    )
)

# Merge agent counts
df_market = df_market.merge(agent_counts, on="market_name")

# Compute CORRECT per-agent metrics
df_market["avg_income_per_agent"] = (
    df_market["agent_commission_sum"] / df_market["agent_count"]
)

df_market["smiley_rev_per_agent"] = (
    df_market["smiley_revenue_sum"] / df_market["agent_count"]
)

# -----------------------------
# 2. METRICS DICTIONARY
# -----------------------------
metrics = {
    "Total Sales": ("total_sales", "🏆 Top 10 Markets by Total Sales"),
    "Average Sale": ("avg_sale", "📈 Top 10 Markets by Average Sale"),
    "Median Sale": ("median_sale", "📐 Top 10 Markets by Median Sale"),
    "# of Sales": ("num_sales", "🔢 Top 10 Markets by # of Sales"),

    "Agent Commission (Sum)": ("agent_commission_sum", "💼 Top 10 Markets by Agent Commission (Sum)"),
    "Agent Commission (Median)": ("agent_commission_median", "💼 Top 10 Markets by Agent Commission (Median)"),

    "Smiley Revenue (Sum)": ("smiley_revenue_sum", "💰 Top 10 Markets by Smiley Revenue (Sum)"),
    "Smiley Revenue (Median)": ("smiley_revenue_median", "💰 Top 10 Markets by Smiley Revenue (Median)"),

    # NEW METRICS
    "Avg Agent Income per Agent": ("avg_income_per_agent", "👩‍💼 Avg Agent Income per Agent (Market Ranking)"),
    "Avg Smiley Revenue per Agent": ("smiley_rev_per_agent", "🏢 Smiley Revenue per Agent (Market Ranking)"),
}

# -----------------------------
# 3. TOP 10 LISTS FOR EACH METRIC
# -----------------------------
top10_data = {}
orders = {}

for label, (col, _) in metrics.items():
    top10 = df_market.nlargest(10, col)
    order = top10.sort_values(col, ascending=False)["market_name"].tolist()[::-1]
    top10_data[label] = top10.set_index("market_name")[col]
    orders[label] = order

# --------------------------
# 4. Create Figure
# --------------------------
fig = go.Figure()

for i, (label, series) in enumerate(top10_data.items()):
    fig.add_trace(
        go.Bar(
            x=series.loc[orders[label]],
            y=orders[label],
            orientation="h",
            name=label,
            visible=(i == 0)
        )
    )

# --------------------------
# 5. Dropdown Buttons
# --------------------------
buttons = []
for i, (label, (_, title)) in enumerate(metrics.items()):
    visibility = [False] * len(metrics)
    visibility[i] = True

    buttons.append(
        dict(
            label=label,
            method="update",
            args=[
                {"visible": visibility},
                {
                    "title": title,
                    "yaxis": {"categoryorder": "array", "categoryarray": orders[label]}
                }
            ]
        )
    )

# --------------------------
# 6. Final Layout
# --------------------------
fig.update_layout(
    title=list(metrics.values())[0][1],
    height=700,
    xaxis_title="Metric Value",
    yaxis_title="Market",
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            showactive=True,
            x=0.5,
            xanchor="center",
            y=1.20,
            yanchor="top"
        )
    ]
)

fig.show()


In [ ]:
df_sales.columns

Index(['sale_id', 'customer_id', 'sale_date', 'amount', 'year_month',
       'year_month_period', 'year', 'month', 'sale_dow', 'sale_mmddyy',
       'agent_commission', 'smiley_revenue'],
      dtype='object')

In [ ]:
# @title Full Customer + Calls + Sales + Agents + Markets Join
query = """
WITH call_stats AS (
    SELECT
        customer_id,
        COUNT(*) AS total_calls,
        SUM(disposition = 'Talked to Customer') AS talked_to_customer,
        SUM(disposition = 'Voicemail') AS voicemail_count,
        SUM(disposition = 'Disconnected') AS disconnected_count,
        MIN(call_date) AS first_call_date,
        MAX(call_date) AS last_call_date
    FROM df_calls
    GROUP BY customer_id
),
sales_stats AS (
    SELECT
        customer_id,
        COUNT(distinct sale_id) AS num_sales,
        MAX(sale_date) AS last_sale_date,
        SUM(amount) AS total_sale_amount,
        SUM(agent_commission) AS total_agent_commission,
        SUM(smiley_revenue) AS total_smiley_revenue
    FROM df_sales
    GROUP BY customer_id
)
SELECT
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    c.state,
    c.created_at,

    -- Call stats
    CASE WHEN total_calls > 0 THEN 1 ELSE 0 END AS has_calls,
    COALESCE(cs.total_calls, 0) AS total_calls,
    COALESCE(cs.talked_to_customer, 0) AS talked_to_customer,
    COALESCE(cs.voicemail_count, 0) AS voicemail_count,
    COALESCE(cs.disconnected_count, 0) AS disconnected_count,
    cs.first_call_date,
    cs.last_call_date,

    -- Sales stats
    CASE WHEN num_sales > 0 THEN 'yes' ELSE 'no' END AS has_sales,
    CASE WHEN talked_to_customer > 0 THEN 'yes' ELSE 'no' END AS has_successful_call,
    COALESCE(ss.num_sales, 0) AS num_sales,
    ss.last_sale_date,
    COALESCE(ss.total_sale_amount, 0) AS total_sale_amount,
    COALESCE(ss.total_agent_commission, 0) AS total_agent_commission,
    COALESCE(ss.total_smiley_revenue, 0) AS total_smiley_revenue,

    -- Agent info
    a.agent_id,
    a.first_name || ' ' || a.last_name AS agent_name,

    -- Market info
    m.common_name AS market_name,
    m.state AS market_state,
    m.population,
    m.days_on_market_median,
    m.price_closed_median,
    m.listings_closed_count

FROM df_customers c
LEFT JOIN call_stats cs ON c.customer_id = cs.customer_id
LEFT JOIN sales_stats ss ON c.customer_id = ss.customer_id
LEFT JOIN df_agents a ON c.assigned_agent = a.agent_id
LEFT JOIN df_markets m ON a.market_id = m.market_id
"""
df_customer_analysis = duckdb.query(query).df()
df_customer_analysis.head()


,customer_id,customer_name,state,created_at,has_calls,total_calls,talked_to_customer,voicemail_count,disconnected_count,first_call_date,...,total_agent_commission,total_smiley_revenue,agent_id,agent_name,market_name,market_state,population,days_on_market_median,price_closed_median,listings_closed_count
0,7,Mitsue Tollner,OH,2025-01-01 06:21:00,1,5,3.0,2.0,0.0,2025-01-01 06:21:00,...,5619.60,2809.80,arjun_ramirez_306,Arjun Ramirez,"Cleveland, OH",OH,2171978,29,243539.0,2292
1,15,Cammy Albares,OH,2025-01-01 08:45:00,1,5,3.0,2.0,0.0,2025-01-01 08:45:00,...,10018.08,5009.04,ali_hall_3,Ali Hall,"Cleveland, OH",OH,2171978,29,243539.0,2292
2,17,Meaghan Garufi,OH,2025-01-01 09:10:00,1,4,3.0,1.0,0.0,2025-01-01 09:10:00,...,4956.96,2478.48,arjun_ramirez_306,Arjun Ramirez,"Cleveland, OH",OH,2171978,29,243539.0,2292
3,19,Yuki Whobrey,TX,2025-01-01 09:39:00,1,2,2.0,0.0,0.0,2025-01-01 09:39:00,...,7984.14,3992.07,asha_johnson_74,Asha Johnson,"Dallas, TX",TX,7807555,64,408386.0,6148
4,20,Fletcher Flosi,NY,2025-01-01 09:44:00,1,4,2.0,1.0,1.0,2025-01-01 09:44:00,...,4757.70,2378.85,aya_gonzalez_760,Aya Gonzalez,"Buffalo, NY",NY,1161385,12,290000.0,1026


In [ ]:
#@title 📞 Customer Funnel: Calls → Conversations → Sales (with Percentages)

import plotly.express as px
import pandas as pd

# Base counts
calls = df_customer_analysis["has_calls"].sum()
talked = (df_customer_analysis["talked_to_customer"] > 0).sum()
sales = df_customer_analysis["has_sales"].eq("yes").sum()

# Percentages
pct_calls_to_talked = talked / calls * 100 if calls > 0 else 0
pct_talked_to_sales = sales / talked * 100 if talked > 0 else 0
pct_calls_to_sales = sales / calls * 100 if calls > 0 else 0

# Build DF
funnel_df = pd.DataFrame({
    "Stage": [
        f"Customers Who Reached Out\n({calls:,})",
        f"Had a Conversation\n({talked:,})\n{pct_calls_to_talked:.1f}% of Calls",
        f"Converted to Sale\n({sales:,})\n{pct_talked_to_sales:.1f}% of Conversations"
    ],
    "Count": [calls, talked, sales],
})

# Plot
fig = px.funnel(
    funnel_df,
    x="Count",
    y="Stage",
    title="📞 Customer Funnel: Calls → Conversations → Sales<br><sup>With Stage Conversion Percentages</sup>",
)

fig.update_layout(
    plot_bgcolor="white",
    paper_bgcolor="white",
    font=dict(size=14),
)

fig.show()

print("📊 Funnel Conversion Summary")
print(f"• Calls → Conversations: {pct_calls_to_talked:.1f}%")
print(f"• Conversations → Sales: {pct_talked_to_sales:.1f}%")
print(f"• Calls → Sales: {pct_calls_to_sales:.1f}%")


📊 Funnel Conversion Summary
• Calls → Conversations: 86.5%
• Conversations → Sales: 32.8%
• Calls → Sales: 28.3%


In [ ]:
#@title 🏆 Conversion Rate by Market

df_conv = (
    df_customer_analysis.groupby("market_name", as_index=False)
    .agg(
        customers=("customer_id", "count"),
        sales=("has_sales", lambda x: (x == "yes").sum())
    )
)

df_conv["conversion_rate"] = df_conv["sales"] / df_conv["customers"]

fig = px.bar(
    df_conv.sort_values("conversion_rate"),
    x="market_name",
    y="conversion_rate",
    title="🏆 Conversion Rate by Market",
)
fig.update_layout(xaxis_tickangle=45, yaxis_tickformat=".0%")
fig.show()


In [ ]:
#@title 👩‍💼 Agent Productivity: Calls vs Sales
df_agent = (
    df_customer_analysis.groupby("agent_name", as_index=False)
    .agg(
        total_calls=("total_calls", "sum"),
        total_sales=("num_sales", "sum")
    )
)

fig = px.scatter(
    df_agent,
    x="total_calls",
    y="total_sales",
    title="👩‍💼 Agent Productivity: Calls vs Sales",
    hover_name="agent_name"
)
fig.show()



In [ ]:
#@title 🏘️ Market Quadrants: Speed vs Pricing
# --- Group by market to avoid duplication ---
df_market_health = (
    df_customer_analysis
    .groupby(["market_name", "market_state"], as_index=False)
    .agg(
        days_on_market_median=("days_on_market_median", "median"),
        price_closed_median=("price_closed_median", "median")
    )
)

# --- Compute medians for quadrant split ---
dom_median = df_market_health["days_on_market_median"].median()
price_median = df_market_health["price_closed_median"].median()

# --- Assign quadrant labels ---
def classify(row):
    if row["days_on_market_median"] <= dom_median and row["price_closed_median"] >= price_median:
        return "🔥 Hot & Expensive"
    elif row["days_on_market_median"] <= dom_median and row["price_closed_median"] < price_median:
        return "⚡ Fast & Affordable"
    elif row["days_on_market_median"] > dom_median and row["price_closed_median"] >= price_median:
        return "💎 Slow but High-Value"
    else:
        return "🐢 Slow & Affordable"

df_market_health["quadrant"] = df_market_health.apply(classify, axis=1)

# --- Quadrant Scatter Plot ---
import plotly.express as px

fig = px.scatter(
    df_market_health,
    x="days_on_market_median",
    y="price_closed_median",
    color="quadrant",
    hover_name="market_name",
    hover_data=["market_state"],
    title="🏘️ Market Quadrants: Speed vs Pricing",
    labels={
        "days_on_market_median": "Median Days on Market (Speed)",
        "price_closed_median": "Median Closed Price ($)"
    },
    # 🔵 Okabe–Ito colorblind-safe palette
    color_discrete_map={
        "🔥 Hot & Expensive": "#E69F00",  # Orange (replaces red)
        "⚡ Fast & Affordable": "#56B4E9",  # Sky blue
        "💎 Slow but High-Value": "#009E73",  # Bluish green
        "🐢 Slow & Affordable": "#999999"  # Gray
    }
)

# Quadrant reference lines
fig.add_vline(x=dom_median, line_dash="dash", line_color="gray")
fig.add_hline(y=price_median, line_dash="dash", line_color="gray")

fig.update_layout(plot_bgcolor="white")
fig.update_yaxes(showgrid=True, gridcolor="#eee")

fig.show()

print("Median DOM (vertical line):", dom_median)
print("Median Price (horizontal line):", price_median)


Median DOM (vertical line): 44.0
Median Price (horizontal line): 395000.0


In [ ]:
#@title 📅 Monthly Call Volume
df_monthly = (
    df_customer_analysis
    .groupby(df_customer_analysis["first_call_date"].dt.to_period("M"))
    ["customer_id"]
    .count()
    .reset_index()
)

df_monthly["first_call_date"] = df_monthly["first_call_date"].dt.to_timestamp()

fig = px.line(
    df_monthly,
    x="first_call_date",
    y="customer_id",
    title="📞 Monthly Customer Call Volume",
    labels={"customer_id": "Call Count", "first_call_date": "Month"},
    markers=True
)

fig.update_layout(
    plot_bgcolor="white",
    xaxis_showgrid=True,
    yaxis_showgrid=True,
)

fig.show()


In [ ]:
# @title 📅 Heatmap: Call Volume by Day-of-Week × Hour-of-Day

import pandas as pd
import plotly.express as px

# --- Make a working copy ---
df_temp = df_customer_analysis.copy()

# --- Extract DOW + hour from first_call_date ---
df_temp = df_temp[df_temp["first_call_date"].notna()].copy()
df_temp["dow"] = df_temp["first_call_date"].dt.day_name()
df_temp["hour"] = df_temp["first_call_date"].dt.hour

# --- Order DOW properly (Mon→Sun) ---
dow_order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
df_temp["dow"] = pd.Categorical(df_temp["dow"], categories=dow_order, ordered=True)

# --- Group for heatmap ---
df_heatmap = (
    df_temp.groupby(["dow","hour"])["customer_id"]
    .count()
    .reset_index(name="call_count")
)

# --- Plot ---
fig = px.imshow(
    df_heatmap.pivot(index="dow", columns="hour", values="call_count"),
    labels={"color": "Call Count"},
    title="📞 Call Volume by Day of Week × Hour of Day",
    aspect="auto",
    color_continuous_scale="Blues"
)

fig.update_layout(
    xaxis_title="Hour of Day",
    yaxis_title="Day of Week",
    plot_bgcolor="white"
)

fig.show()


/tmp/ipython-input-2961379374.py:20: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
# @title 📅 Heatmap: NON-Successful Calls by Day-of-Week × Hour-of-Day

import pandas as pd
import plotly.express as px

# --- Make working copy ---
df_temp = df_customer_analysis.copy()

# --- Filter to NON-successful calls ---
df_temp = df_temp[df_temp["has_successful_call"] == "no"]

# --- Keep only rows with call timestamps ---
df_temp = df_temp[df_temp["first_call_date"].notna()].copy()

# --- Extract DOW + hour ---
df_temp["dow"] = df_temp["first_call_date"].dt.day_name()
df_temp["hour"] = df_temp["first_call_date"].dt.hour

# --- Order DOW properly (Mon → Sun) ---
dow_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
df_temp["dow"] = pd.Categorical(df_temp["dow"], categories=dow_order, ordered=True)

# --- Build heatmap dataframe ---
df_heatmap = (
    df_temp.groupby(["dow", "hour"])["customer_id"]
    .count()
    .reset_index(name="call_count")
)

# --- Plot Heatmap ---
fig = px.imshow(
    df_heatmap.pivot(index="dow", columns="hour", values="call_count"),
    labels={"color": "Non-Successful Calls"},
    title="📅 Non-Successful Call Volume — Day of Week × Hour of Day",
    aspect="auto",
    color_continuous_scale="Reds"   # red for problems / failed calls
)

fig.update_layout(
    xaxis_title="Hour of Day",
    yaxis_title="Day of Week",
    plot_bgcolor="white"
)

fig.show()


/tmp/ipython-input-3950227228.py:25: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
#@title 📞 Successful vs Non-Successful Calls by State (Had Conversation)
# Build grouped data
df_heat_plot = (
    df_customer_analysis
    .groupby(["state", "has_successful_call"])
    .size()
    .reset_index(name="count")
)

# Custom color map
color_map = {
    "yes": "#1f77b4",   # blue
    "no":  "#7f7f7f"    # grey
}

fig = px.bar(
    df_heat_plot,
    x="state",
    y="count",
    color="has_successful_call",
    barmode="group",
    title="📞 Successful vs Non-Successful Calls by State",
    labels={"count": "Call Count", "has_successful_call": "Successful Call?"},
    color_discrete_map=color_map
)

fig.update_layout(
    xaxis_tickangle=45,
    plot_bgcolor="white",
    paper_bgcolor="white",
    yaxis=dict(showgrid=True, gridcolor="#E5E5E5")
)

fig.show()


/tmp/ipython-input-3574817937.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
df_customer_analysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   customer_id             50000 non-null  object        
 1   customer_name           50000 non-null  object        
 2   state                   50000 non-null  category      
 3   created_at              50000 non-null  datetime64[ns]
 4   has_calls               50000 non-null  int32         
 5   total_calls             50000 non-null  int64         
 6   talked_to_customer      50000 non-null  float64       
 7   voicemail_count         50000 non-null  float64       
 8   disconnected_count      50000 non-null  float64       
 9   first_call_date         50000 non-null  datetime64[ns]
 10  last_call_date          50000 non-null  datetime64[ns]
 11  has_sales               50000 non-null  object        
 12  has_successful_call     50000 non-null  object

In [ ]:
#@title 📞 Do Customers With Sales Tend to Have More Calls?
df_temp = df_customer_analysis.copy()
df_temp["has_sales"] = df_temp["has_sales"].astype(str)

import plotly.express as px

fig = px.box(
    df_temp,
    x="has_sales",
    y="talked_to_customer",
    title="📞 Do Customers With Sales Tend to Have More Calls?",
    labels={
        "has_sales": "Has a Sale?",
        "talked_to_customer": "Total Calls"
    },
    points="all"  # optional: show all points
)

fig.update_layout(
    yaxis_title="Number of Calls",
    xaxis_title="Sales Outcome",
)

fig.show()




In [ ]:
#@title 📞 Call Volume — Proportion of Sales vs No-Sales Customers
fig = px.histogram(
    df_temp,
    x="talked_to_customer",
    color="has_sales",
    barmode="relative",  # <- stacked as proportions
    title="📞 Call Volume — Proportion of Sales vs No-Sales Customers",
)

fig.update_layout(
    paper_bgcolor="white",
    plot_bgcolor="white"
)

fig.show()


In [ ]:
#@title ⏱️ Distribution of Time to Sale (Days from First Call to Sale)
# --- Compute Time to Sale ---
df_temp = df_customer_analysis.copy()
df_temp["time_to_sale"] = (
    df_temp["last_sale_date"] - df_temp["first_call_date"]
).dt.days

# --- Box Plot: Time from First Call → Sale ---
fig = px.box(
    df_temp[df_temp["time_to_sale"].notna()],   # include only customers with a sale
    y="time_to_sale",
    title="⏱️ Distribution of Time to Sale (Days from First Call to Sale)",
    labels={"time_to_sale": "Days from First Call to Sale"},
)

fig.update_layout(
    template="simple_white",
    yaxis=dict(gridcolor="lightgray"),
)

fig.show()


In [ ]:
# @title ⏱️ How Long Does It Take to Close a Sale After First Call? (With Percentiles)

import plotly.express as px
import plotly.graph_objects as go

# Filter valid rows
df_nonnull = df_temp[df_temp["time_to_sale"].notna()]

# Compute percentiles
p25 = df_nonnull["time_to_sale"].quantile(0.25)
p50 = df_nonnull["time_to_sale"].quantile(0.50)
p75 = df_nonnull["time_to_sale"].quantile(0.75)
p95 = df_nonnull["time_to_sale"].quantile(0.95)

# Create histogram using original px colors
fig = px.histogram(
    df_nonnull,
    x="time_to_sale",
    nbins=30,
    opacity=0.7,
    title="⏱️ How Long Does It Take to Close a Sale After First Call?",
    labels={"time_to_sale": "Days from First Call to Sale"},
)

# Convert to go.Figure so we can add lines
fig = go.Figure(fig)

# Add percentile lines
percentiles = {
    "25th": p25,
    "Median": p50,
    "75th": p75,
    "95th": p95
}

colors = {
    "25th": "#5DADE2",
    "Median": "#2E86C1",
    "75th": "#1B4F72",
    "95th": "#154360"
}

for label, value in percentiles.items():
    fig.add_vline(x=value, line_dash="dash", line_color=colors[label], line_width=3)
    fig.add_annotation(
        x=value,
        y=1,
        yref="paper",
        text=f"{label}",
        showarrow=False,
        font=dict(color=colors[label], size=12)
    )

# Layout formatting
fig.update_layout(
    bargap=0.02,
    plot_bgcolor="white",
    width=1000,
    height=550,
    xaxis_title="⏱️ Days from First Call to Sale",
    yaxis_title="Frequency",
)

# Gridlines
fig.update_yaxes(showgrid=True, gridcolor="#EBF5FB")
fig.update_xaxes(showgrid=False)

fig.show()

print(f"25th percentile: {p25:.1f} days")
print(f"Median: {p50:.1f} days")
print(f"75th percentile: {p75:.1f} days")
print(f"95th percentile: {p95:.1f} days")


25th percentile: 3.0 days
Median: 5.0 days
75th percentile: 8.0 days
95th percentile: 12.0 days


In [ ]:
#@title ⏱️ Time to Sale Distribution by Market
df_valid = df_temp[df_temp["time_to_sale"].notna()]

fig = px.box(
    df_valid,
    x="market_name",
    y="time_to_sale",
    points="all",
    title="⏱️ Time to Sale Distribution by Market",
    labels={"time_to_sale": "Days from First Call to Sale"},
)
fig.update_layout(xaxis_tickangle=45, height=600)
fig.show()


# 🔹 Modeling Stage

##  Agent Performance Score (APS) Methodology

The APS is a single standardized index blending productivity, efficiency, and financial impact, designed to fairly evaluate agents across our 50 national markets.

### 1. Normalization Strategy (Ensuring Fairness)

All five metrics are normalized to a **0–1 scale**.

* **Market-Tiered Normalization:** **Revenue Generated** and **Median Sale Value** are normalized only against the min/max values of the agent's specific market. This is crucial to ensure agents in low-value markets can achieve a high score relative to their local peers.
* **Global Normalization:** **Number of Sales**, **Conversion Rate**, and **Average Time to Sale** are normalized nationally, as these behavioral metrics are universally comparable. *Time to Sale* is **inverse-scored** (faster = higher score).

### 2. Weighted Combination (National Growth Focus)

The weights reflect the company's priority on **national expansion and volume/impact**:

* **Revenue Generated:** **30%** (Core financial impact)
* **Number of Sales:** **30%** (Volume and market penetration)
* **Conversion Rate:** **20%** (Efficiency and skill in lead utilization)
* **Median Sale Value:** **10%** (Market-fair deal quality)
* **Average Time to Sale:** **10%** (Pipeline speed and efficiency)

The resulting APS identifies top agents based on a balance of **volume** and **efficiency**, while maintaining fairness across the varied economic realities of our national markets.

In [ ]:
# @title ⭐ Agent Performance Score (APS)

import duckdb
import pandas as pd
import numpy as np

# NOTE: df_customers, df_agents, df_markets, and df_sales are assumed to be loaded
#       as pandas DataFrames (e.g., from CSV files) before this script runs.

# --- GLOBAL NORMALIZATION FUNCTION (Used for nationally comparable metrics) ---
def norm_global(s, inverse=False):
    """Normalizes a series globally across the entire dataset."""
    s = (s - s.min()) / (s.max() - s.min() + 1e-6)
    return 1 - s if inverse else s

# --- TIERED NORMALIZATION FUNCTION (Used for market-dependent metrics) ---
def norm_market_tier(df, metric, inverse=False):
    """Normalizes a metric within each 'market_name' group."""
    # This uses .transform, which is generally safe from SwaCW
    df[f"n_{metric}"] = df.groupby('market_name')[metric].transform(
        lambda x: (x - x.min()) / (x.max() - x.min() + 1e-6)
    )
    return 1 - df[f"n_{metric}"] if inverse else df[f"n_{metric}"]


# ----------------------------------------------------------------------
# ---- STEP 1: Build agent-level aggregates using DuckDB SQL (FIXED) ----
# ----------------------------------------------------------------------

query = """
SELECT
    a.agent_id,
    a.first_name || ' ' || a.last_name AS agent_name,
    m.common_name AS market_name,

    -- Productivity/Volume
    COUNT(s.sale_id) AS num_sales,
    SUM(s.smiley_revenue) AS total_revenue,
    MEDIAN(s.amount) AS median_sale,

    -- Time to sale (Coalesce used for safety, though AVG will handle NULLs where num_sales=0)
    AVG(DATEDIFF('day', c.created_at, s.sale_date)) AS avg_time_to_sale,

    -- Conversion: # sales / # customers assigned
    CAST(COUNT(s.sale_id) AS DOUBLE) / COUNT(c.customer_id) AS conversion_rate

FROM df_customers c
LEFT JOIN df_agents a ON c.assigned_agent = a.agent_id
LEFT JOIN df_markets m ON a.market_id = m.market_id
LEFT JOIN df_sales s ON s.customer_id = c.customer_id

GROUP BY a.agent_id, a.first_name, a.last_name, market_name
"""

# FIX 1: Explicitly copy the DataFrame returned from DuckDB
df_agent = duckdb.query(query).df().copy()

# Handle potential NaN values resulting from agents with 0 sales/customers
# FIX 2: Replace inplace=True with direct assignment (=)
# For avg_time_to_sale, if an agent has no sales, we penalize them by assigning the max time to sale.
df_agent['avg_time_to_sale'] = df_agent['avg_time_to_sale'].fillna(df_agent['avg_time_to_sale'].max())

# FIX 3: Replace inplace=True with direct assignment using the mapping method
fill_values = {
    'total_revenue': 0,
    'median_sale': 0,
    'conversion_rate': 0
}
df_agent.fillna(fill_values, inplace=True) # NOTE: Using inplace here is generally okay on a base DF, but for ultimate safety, use:
# df_agent = df_agent.fillna(fill_values)


# ----------------------------------------------------------------------
# ---- STEP 2: Normalize KPIs 0–1 (Tiered vs. Global) ----
# ----------------------------------------------------------------------
# (No changes needed here as this uses direct column assignment and .transform, which are safe)

# Tiered Normalization (Relative to Market) for Value/Impact Metrics:
df_agent["n_revenue"]     = norm_market_tier(df_agent, "total_revenue")
df_agent["n_median_sale"] = norm_market_tier(df_agent, "median_sale")

# Global Normalization (Relative to Nation) for Behavioral/Efficiency Metrics:
df_agent["n_sales"]        = norm_global(df_agent["num_sales"])
df_agent["n_conversion"]   = norm_global(df_agent["conversion_rate"])
df_agent["n_time_to_sale"] = norm_global(df_agent["avg_time_to_sale"], inverse=True)


# ----------------------------------------------------------------------
# ---- STEP 3: Weighted Agent Performance Score (APS) ----
# ----------------------------------------------------------------------
# Using the national growth focused weights (30/30/10/20/10)
df_agent["APS"] = (
    df_agent["n_revenue"]     * 0.30 +
    df_agent["n_sales"]       * 0.30 +
    df_agent["n_median_sale"] * 0.10 +
    df_agent["n_conversion"]  * 0.20 +
    df_agent["n_time_to_sale"]* 0.10
)


# ----------------------------------------------------------------------
# ---- STEP 4: Get the BEST agent per market ----
# ----------------------------------------------------------------------

df_best_agents = (
    df_agent.sort_values("APS", ascending=False)
            .groupby("market_name")
            .head(1)
            .reset_index(drop=True)
)

df_best_agents.head()

,agent_id,agent_name,market_name,num_sales,total_revenue,median_sale,avg_time_to_sale,conversion_rate,n_total_revenue,n_revenue,n_median_sale,n_sales,n_conversion,n_time_to_sale,APS
0,daniel_lopez_224,Daniel Lopez,"Phoenix, AZ",29,148294.19,521605.0,6.551724,0.349398,1.0,1.0,0.901464,1.000000,0.577802,0.442087,0.849915
1,michael_hall_387,Michael Hall,"Portland, OR",28,136705.72,491495.5,6.000000,0.388889,1.0,1.0,0.750748,0.961538,0.662844,0.512821,0.847387
2,dorothy_mohamed_578,Dorothy Mohamed,"Milwaukee, WI",24,121035.73,523837.0,5.500000,0.461538,1.0,1.0,0.794077,0.807692,0.819290,0.576923,0.843266
3,ali_hall_3,Ali Hall,"Cleveland, OH",26,129712.20,511245.5,6.115385,0.426230,1.0,1.0,0.716823,0.884615,0.743255,0.498028,0.835521
4,jason_davis_983,Jason Davis,"Minneapolis, MN",24,122703.01,523532.0,6.875000,0.406780,1.0,1.0,0.970238,0.807692,0.701371,0.400641,0.819670


In [ ]:
# -------------------------------------------
#@title 📋 Best Agent per Market — Table Version
# -------------------------------------------

# Select the columns you want in the table
df_best_agent_table = df_best_agents[[
    "market_name",
    "agent_name",
    "agent_id",
    "median_sale",
    "num_sales",
    "total_revenue",
    "conversion_rate",
    "avg_time_to_sale",
    "APS"
]].copy()

# Sort by APS descending (best at top)
df_best_agent_table = df_best_agent_table.sort_values("APS", ascending=False)

# Format APS for clarity
df_best_agent_table["APS"] = df_best_agent_table["APS"].round(3)

df_best_agent_table.head(10)   # show first 20 markets

,market_name,agent_name,agent_id,median_sale,num_sales,total_revenue,conversion_rate,avg_time_to_sale,APS
0,"Phoenix, AZ",Daniel Lopez,daniel_lopez_224,521605.0,29,148294.19,0.349398,6.551724,0.850
1,"Portland, OR",Michael Hall,michael_hall_387,491495.5,28,136705.72,0.388889,6.000000,0.847
2,"Milwaukee, WI",Dorothy Mohamed,dorothy_mohamed_578,523837.0,24,121035.73,0.461538,5.500000,0.843
3,"Cleveland, OH",Ali Hall,ali_hall_3,511245.5,26,129712.20,0.426230,6.115385,0.836
4,"Minneapolis, MN",Jason Davis,jason_davis_983,523532.0,24,122703.01,0.406780,6.875000,0.820
5,"Atlanta, GA",Arjun Robinson,arjun_robinson_452,462604.5,24,111109.84,0.545455,6.166667,0.819
6,"San Antonio, TX",Jason Mitchell,jason_mitchell_515,574107.0,22,124044.97,0.407407,5.909091,0.812
7,"Charlotte, NC",Anthony Williams,anthony_williams_805,538084.0,27,140756.99,0.313953,7.000000,0.804
8,"Cincinnati, OH",Dorothy Gonzalez,dorothy_gonzalez_346,440902.0,28,126905.19,0.424242,5.321429,0.802
9,"Kansas City, MO",Dorothy Martinez,dorothy_martinez_83,516222.0,22,111035.48,0.400000,3.909091,0.799


In [136]:
df_best_agent_table.to_csv("best_agents_table.csv", index=False)

In [ ]:
# @title ⭐ Lollipop Chart — Best Agent Per Market (APS Score)

import plotly.graph_objects as go

df_plot = df_best_agents.sort_values("APS", ascending=True).copy()

fig = go.Figure()

# --- 1. Add the line segment for each market ---
for i, row in df_plot.iterrows():
    fig.add_shape(
        type="line",
        x0=0,
        y0=row["market_name"],
        x1=row["APS"],
        y1=row["market_name"],
        line=dict(color="#AAB7B8", width=2)
    )

# --- 2. Add the dot at the APS score ---
fig.add_trace(go.Scatter(
    x=df_plot["APS"],
    y=df_plot["market_name"],
    mode="markers+text",
    marker=dict(color="#1F77B4", size=12),
    text=df_plot["agent_name"] + " (" + df_plot["agent_id"] + ")",
    textposition="middle right",
    name="APS Score"
))

# --- Layout ---
fig.update_layout(
    title="⭐ Best Agent Per Market (APS Score)",
    xaxis_title="APS Score",
    yaxis_title="Market",
    height=900,
    plot_bgcolor="white",
    xaxis=dict(showgrid=True, gridcolor="#EBEBEB"),
    yaxis=dict(showgrid=False)
)

fig.show()


In [ ]:
#@title 🎯 Agent Performance Quadrants: Conversion vs Time-to-Sale
#-------------------------------------------
# AGENT QUADRANT: Conversion vs Time to Sale
# -------------------------------------------

import plotly.express as px
import numpy as np

df_quadrant = df_best_agents.copy()

# --- Compute medians for quadrant split ---
conv_median = df_quadrant["conversion_rate"].median()
tts_median  = df_quadrant["avg_time_to_sale"].median()

# --- Assign quadrant labels ---
def classify_agent(row):
    if row["conversion_rate"] >= conv_median and row["avg_time_to_sale"] <= tts_median:
        return "🔥 High Conversion + Fast Closer"
    elif row["conversion_rate"] >= conv_median and row["avg_time_to_sale"] > tts_median:
        return "💎 High Conversion + Slow Closer"
    elif row["conversion_rate"] < conv_median and row["avg_time_to_sale"] <= tts_median:
        return "⚡ Low Conversion + Fast Closer"
    else:
        return "🐢 Low Conversion + Slow Closer"

df_quadrant["quadrant"] = df_quadrant.apply(classify_agent, axis=1)

# --- Colorblind-friendly palette (retain emojis) ---
color_map = {
    "🔥 High Conversion + Fast Closer": "#D55E00",   # orange
    "💎 High Conversion + Slow Closer": "#0072B2",   # blue
    "⚡ Low Conversion + Fast Closer": "#009E73",   # green
    "🐢 Low Conversion + Slow Closer": "#999999"    # gray
}

# --- Scatter Plot ---
fig = px.scatter(
    df_quadrant,
    x="conversion_rate",
    y="avg_time_to_sale",
    color="quadrant",
    hover_name="agent_name",
    hover_data=["market_name", "num_sales", "median_sale"],
    title="🎯 Agent Performance Quadrants: Conversion vs Time-to-Sale",
    labels={
        "conversion_rate": "Conversion Rate",
        "avg_time_to_sale": "Avg Time to Close (days)"
    },
    color_discrete_map=color_map
)

# --- Add quadrant lines ---
fig.add_vline(x=conv_median, line_dash="dash", line_color="black")
fig.add_hline(y=tts_median, line_dash="dash", line_color="black")

# --- Layout ---
fig.update_layout(
    plot_bgcolor="white",
    xaxis_showgrid=True, yaxis_showgrid=True,
    xaxis_gridcolor="#EEEEEE", yaxis_gridcolor="#EEEEEE",
)

fig.show()

print("Median Conversion Rate:", conv_median)
print("Median Time to Sale:", tts_median)


Median Conversion Rate: 0.3774134790528233
Median Time to Sale: 6.18859649122807


In [ ]:
# -------------------------------------
#@title 🧮 Quadrant Scatterplot — Revenue vs Conversion Rate
# -------------------------------------

import plotly.express as px

df = df_best_agents.copy()

# --- Compute medians for quadrant splits ---
rev_median = df["total_revenue"].median()
conv_median = df["conversion_rate"].median()

# --- Assign quadrant labels ---
def classify(row):
    if row["total_revenue"] >= rev_median and row["conversion_rate"] >= conv_median:
        return "🌟 High Revenue • High Conversion"
    elif row["total_revenue"] >= rev_median and row["conversion_rate"] < conv_median:
        return "💰 High Revenue • Low Conversion"
    elif row["total_revenue"] < rev_median and row["conversion_rate"] >= conv_median:
        return "📈 Low Revenue • High Conversion"
    else:
        return "🪫 Low Revenue • Low Conversion"

df["quadrant"] = df.apply(classify, axis=1)

# --- Colorblind-friendly palette ---
color_map = {
    "🌟 High Revenue • High Conversion": "#0072B2",  # blue
    "💰 High Revenue • Low Conversion":  "#E69F00",  # orange
    "📈 Low Revenue • High Conversion":  "#009E73",  # green
    "🪫 Low Revenue • Low Conversion":    "#D55E00"   # vermillion
}

# --- Scatterplot ---
fig = px.scatter(
    df,
    x="total_revenue",
    y="conversion_rate",
    color="quadrant",
    hover_name="agent_name",
    hover_data=["market_name", "num_sales", "avg_time_to_sale"],
    title="🧭 Agent Market Positioning — Revenue vs Conversion Rate",
    labels={
        "total_revenue": "Total Revenue Generated ($)",
        "conversion_rate": "Lead → Sale Conversion Rate"
    },
    color_discrete_map=color_map
)

# --- Quadrant reference lines ---
fig.add_vline(x=rev_median, line_dash="dash", line_color="gray")
fig.add_hline(y=conv_median, line_dash="dash", line_color="gray")

fig.update_layout(plot_bgcolor="white")
fig.update_yaxes(showgrid=True, gridcolor="#eee")

fig.show()

print("Median Revenue:", rev_median)
print("Median Conversion Rate:", conv_median)


Median Revenue: 106463.59
Median Conversion Rate: 0.3774134790528233


In [ ]:
# ===============================
# @title ✅ Quadrant Chart: Revenue vs Avg Time to Sale
# ===============================

df = df_best_agents.copy()

# --- Compute medians for quadrants ---
rev_median = df["total_revenue"].median()
tts_median = df["avg_time_to_sale"].median()

# --- Quadrant classification ---
def classify_tts(row):
    if row["total_revenue"] >= rev_median and row["avg_time_to_sale"] <= tts_median:
        return "🚀 High Revenue • Fast Closing"
    elif row["total_revenue"] >= rev_median and row["avg_time_to_sale"] > tts_median:
        return "💰 High Revenue • Slow Closing"
    elif row["total_revenue"] < rev_median and row["avg_time_to_sale"] <= tts_median:
        return "⚡ Low Revenue • Fast Closing"
    else:
        return "🐢 Low Revenue • Slow Closing"

df["quadrant_tts"] = df.apply(classify_tts, axis=1)

# --- Colorblind-friendly palette ---
colors_tts = {
    "🚀 High Revenue • Fast Closing": "#1B9E77",   # green
    "💰 High Revenue • Slow Closing": "#D95F02",   # orange
    "⚡ Low Revenue • Fast Closing": "#7570B3",     # purple
    "🐢 Low Revenue • Slow Closing": "#BDBDBD"      # grey
}

fig = px.scatter(
    df,
    x="total_revenue",
    y="avg_time_to_sale",
    color="quadrant_tts",
    color_discrete_map=colors_tts,
    hover_name="agent_name",
    hover_data=["market_name", "num_sales"],
    title="⏱️ Agent Quadrants — Revenue vs Avg Time to Sale",
    labels={
        "total_revenue": "Total Revenue ($)",
        "avg_time_to_sale": "Average Time to Sale (Days)"
    }
)

# quadrant lines
fig.add_vline(x=rev_median, line_dash="dash", line_color="gray")
fig.add_hline(y=tts_median, line_dash="dash", line_color="gray")

fig.update_layout(
    plot_bgcolor="white",
    xaxis_showgrid=True,
    yaxis_showgrid=True,
    xaxis_gridcolor="#eee",
    yaxis_gridcolor="#eee"
)

fig.show()

print("Revenue median:", rev_median)
print("Avg Time-to-Sale median:", tts_median)


Revenue median: 106463.59
Avg Time-to-Sale median: 6.18859649122807



## 📊 Market Performance Score (MPS) — Explanation

To compare markets fairly—even though they differ in size, sales volume, and local dynamics—I created a **Market Performance Score (MPS)** that combines multiple normalized indicators into a single, comparable metric.

Each component reflects a different dimension of market performance:

* **Revenue (30%)** – Prioritized because it represents direct financial impact and overall profitability of the market.
* **Sales Volume (15%)** – Still important, but down-weighted to avoid over-rewarding very large markets where high revenue and high volume overlap.
* **Conversion Rate (20%)** – Measures how effectively the market turns leads into closed deals.
* **Speed to Sale (10%)** – Faster markets close more efficiently and reduce operational friction.
* **Median Sale Value (10%)** – Captures market quality and typical price point.
* **Population Size (10%)** – Represents market scale and growth potential.
* **Days on Market (5%)** – Included with a lighter weight since it is an external condition rather than a performance-driven one.

To ensure fairness, all inputs were **normalized to a 0–1 scale**, so markets are evaluated relative to each other, not by raw values. This removes biases from scale differences (e.g., large vs. small markets) and allows all metrics to contribute proportionally.

The resulting MPS gives a **holistic, balanced indicator** of each market’s true performance—blending financial results, customer outcomes, efficiency, speed, and growth potential. Markets can then be ranked, compared, and segmented consistently using this unified score.



In [ ]:
# @title ⭐ Market Performance Score (MPS)

import duckdb
import pandas as pd
import numpy as np


# ------------------------------------------------------------
# 1. GLOBAL NORMALIZATION FUNCTION
# ------------------------------------------------------------
def norm_global(series, inverse=False):
    """Normalize a metric 0–1 across all markets nationally."""
    s = (series - series.min()) / (series.max() - series.min() + 1e-6)
    return 1 - s if inverse else s

# NOTE: norm_tier is not used for the final MPS but kept for reference.


# ------------------------------------------------------------
# 2. BUILD MARKET-LEVEL METRICS (CRITICAL SQL JOIN FIX APPLIED)
# ------------------------------------------------------------
query_market = """
SELECT
    m.market_id,
    m.common_name     AS market_name,
    m.state           AS market_state,

    -- External Market Health Data
    m.population,
    m.days_on_market_median,
    m.price_closed_median,
    m.listings_closed_count,
    m.listings_price_decrease_count,

    -- INTERNAL SMILEY PERFORMANCE (Aggregated per Market - No join to df_agents)
    COUNT(s.sale_id)                    AS num_sales,
    SUM(s.smiley_revenue)               AS total_revenue,
    MEDIAN(s.amount)                    AS median_sale,

    -- CUSTOMER CONVERSION (market-level)
    -- Counts all customers in the market (c.customer_id) vs. customers with sales (s.sale_id)
    CAST(COUNT(s.sale_id) AS DOUBLE) / COUNT(c.customer_id) AS conversion_rate,

    -- SPEED OF SUCCESS (average)
    AVG(DATEDIFF('day', c.created_at, s.sale_date)) AS avg_time_to_sale

FROM df_markets m
LEFT JOIN df_customers c  ON c.market_id = m.market_id
-- CRITICAL FIX: Removed join to df_agents to prevent sales/revenue double-counting.
LEFT JOIN df_sales s      ON s.customer_id = c.customer_id

GROUP BY ALL
"""

df_market = duckdb.query(query_market).df().copy()

# Handle missing values (Ensuring Safety and Setting Penalties/Zeros)
# Fill NaN time to sale with the max national time (penalty for zero sales)
df_market["avg_time_to_sale"] = df_market["avg_time_to_sale"].fillna(df_market["avg_time_to_sale"].max())
# Fill other internal performance metrics (num_sales, revenue, conversion) with 0 for markets with zero activity
df_market = df_market.fillna(0)


# ------------------------------------------------------------
# 3. NORMALIZE METRICS (0–1)
# ------------------------------------------------------------

# Internal Performance Metrics (Higher weights)
df_market["n_revenue"]       = norm_global(df_market["total_revenue"])
df_market["n_sales"]         = norm_global(df_market["num_sales"])
df_market["n_median_sale"]   = norm_global(df_market["median_sale"])
df_market["n_conversion"]    = norm_global(df_market["conversion_rate"])
df_market["n_time_speed"]    = norm_global(df_market["avg_time_to_sale"], inverse=True)

# External Market Health/Scale Metrics (Lower weights)
df_market["n_dom"]           = norm_global(df_market["days_on_market_median"], inverse=True)
df_market["n_price_cuts"]    = norm_global(df_market["listings_price_decrease_count"], inverse=True)
df_market["n_population"]    = norm_global(np.log1p(df_market["population"]))


# ------------------------------------------------------------
# 4. MARKET PERFORMANCE SCORE (MPS) - STRATEGIC WEIGHTING
# ------------------------------------------------------------

# Weights adjusted to reward Internal Execution, Market Potential, and reduce bias from external factors.
df_market["MPS"] = (
    df_market["n_revenue"]     * 0.30 +   # Increased: Profitability is top priority.
    df_market["n_sales"]       * 0.15 +   # Reduced: Volume is covered, prevents too much revenue/sales overlap.
    df_market["n_conversion"]  * 0.20 +   # Customer success/efficiency.
    df_market["n_time_speed"]  * 0.10 +   # Speed of closing.
    df_market["n_median_sale"] * 0.10 +   # Value profile.
    df_market["n_population"]  * 0.10 +   # Increased: Market scale/growth potential.
    df_market["n_dom"]         * 0.05     # Reduced: External market factor (less controllable).
    # Removed n_price and n_price_cuts to simplify score and remove external bias
)

# Sort final ranking
df_market = df_market.sort_values("MPS", ascending=False)

df_market.head()

,market_id,market_name,market_state,population,days_on_market_median,price_closed_median,listings_closed_count,listings_price_decrease_count,num_sales,total_revenue,...,avg_time_to_sale,n_revenue,n_sales,n_median_sale,n_conversion,n_time_speed,n_dom,n_price_cuts,n_population,MPS
41,seattle_wa,"Seattle, WA",WA,4021467,21,832871.0,3716,3986,323,1583078.81,...,6.343653,0.749238,1.000000,0.491101,0.866251,0.500941,8.875000e-01,0.727471,0.439487,0.735550
37,grand_rapids_mi,"Grand Rapids, MI",MI,1154320,14,355500.0,1342,1000,314,1561741.94,...,6.226115,0.712115,0.895349,0.636451,0.846023,0.642259,9.750000e-01,0.960243,0.000000,0.693762
43,atlanta_ga,"Atlanta, GA",GA,6176937,60,395000.0,6699,10778,303,1474709.97,...,6.099010,0.560691,0.767442,0.474883,0.948010,0.795079,4.000000e-01,0.198004,0.590606,0.678982
17,miami_fl,"Miami, FL",FL,6138876,92,560000.0,7102,10914,300,1499323.21,...,6.010000,0.603514,0.732558,0.854487,0.649085,0.902097,1.250000e-08,0.187403,0.588430,0.655256
11,washington_dc,"Washington, DC",DC,6263796,45,580748.0,5869,5198,305,1479123.71,...,6.413115,0.568370,0.790698,0.230940,0.999987,0.417427,5.875000e-01,0.632990,0.595523,0.642877


In [135]:
# @title ⭐ Lollipop Chart — Markets by MPS (Market Performance Score)

import plotly.graph_objects as go

# --- Sort markets by MPS (ascending so chart flows top→bottom) ---
df_plot = df_market.sort_values("MPS", ascending=True).copy()

fig = go.Figure()

# --- 1. Add grey line for each market (base of the lollipop) ---
for i, row in df_plot.iterrows():
    fig.add_shape(
        type="line",
        x0=0,
        y0=row["market_name"],
        x1=row["MPS"],
        y1=row["market_name"],
        line=dict(color="#B0B8C1", width=2)   # colorblind-friendly grey
    )

# --- 2. Add the dot at the MPS position ---
fig.add_trace(go.Scatter(
    x=df_plot["MPS"],
    y=df_plot["market_name"],
    mode="markers+text",

    # Colorblind-safe blue marker
    marker=dict(color="#1f78b4", size=12),

    # Market label (name + state)
    text=df_plot["market_name"] + " (" + df_plot["market_state"] + ")",
    textposition="middle right",
    name="MPS Score"
))

# --- Layout ---
fig.update_layout(
    title="⭐ Markets by Market Performance Score (MPS) — Lollipop Chart",
    xaxis_title="Market Performance Score (0–1)",
    yaxis_title="Market",

    height=950,
    plot_bgcolor="white",

    xaxis=dict(showgrid=True, gridcolor="#EBEBEB", zeroline=False),
    yaxis=dict(showgrid=False),

    margin=dict(l=180)  # more room for long market names
)

fig.show()


In [ ]:
# @title 🏆 Podium Chart — Top 5 Markets by MPS (Blue/Purple Palette)

import plotly.graph_objects as go

# --- Get top 5 ---
df_podium = df_market.sort_values("MPS", ascending=False).head(5).copy()

# --- Add ranking 1–5 ---
df_podium["Rank"] = range(1, 6)

# --- Blue / Purple colorblind-safe palette ---
colors = [
    "#4C72B0",  # blue
    "#5F9ED1",  # lighter blue
    "#B39CD0",  # lavender
    "#8E6BBE",  # purple
    "#6A51A3"   # deep purple
]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_podium["Rank"],
    y=df_podium["MPS"],
    text=df_podium["market_name"] + "<br>MPS: " + df_podium["MPS"].round(3).astype(str),
    textposition="inside",
    marker_color=colors,
    width=[0.55]*5
))

# ---- Layout ----
fig.update_layout(
    title="🏆 Top 5 Markets — Podium Ranking by Market Performance Score (MPS)",
    xaxis_title="Rank (1 = Best)",
    yaxis_title="Market Performance Score",
    xaxis=dict(
        tickmode="array",
        tickvals=[1,2,3,4,5],
        ticktext=["1st","2nd","3rd","4th","5th"],
        showgrid=False
    ),
    yaxis=dict(showgrid=True, gridcolor="#E5E7E9"),
    plot_bgcolor="white",
    height=550
)

fig.show()


In [ ]:
# @title 🧭 Market Quadrants: MPS vs Population
import numpy as np
import plotly.express as px

# median splits → quadrant boundaries
mps_median = df_market["MPS"].median()
pop_median = df_market["population"].median()

# quadrant assignment
def classify_market(row):
    if row["MPS"] >= mps_median and row["population"] >= pop_median:
        return "🌎 High MPS & High Population (Core Markets)"
    elif row["MPS"] >= mps_median and row["population"] < pop_median:
        return "🚀 High MPS & Low Population (High-Growth Niches)"
    elif row["MPS"] < mps_median and row["population"] >= pop_median:
        return "⚠️ Low MPS & High Population (Underperforming Giants)"
    else:
        return "📉 Low MPS & Low Population (Low-Priority)"

df_market["quadrant"] = df_market.apply(classify_market, axis=1)

# colorblind-friendly palette
color_map = {
    "🌎 High MPS & High Population (Core Markets)": "#1B9E77",
    "🚀 High MPS & Low Population (High-Growth Niches)": "#7570B3",
    "⚠️ Low MPS & High Population (Underperforming Giants)": "#D95F02",
    "📉 Low MPS & Low Population (Low-Priority)": "#999999",
}

# scatter plot
fig = px.scatter(
    df_market,
    x="population",
    y="MPS",
    color="quadrant",
    color_discrete_map=color_map,
    hover_name="market_name",
    hover_data=["market_state", "num_sales", "total_revenue", "conversion_rate"],
    #size="total_revenue",
    #size_max=35,
    title="🧭 Market Quadrants: Market Performance Score vs Population",
    labels={
        "population": "Market Population",
        "MPS": "Market Performance Score"
    }
)

# quadrant lines
fig.add_vline(x=pop_median, line_dash="dash", line_color="gray")
fig.add_hline(y=mps_median, line_dash="dash", line_color="gray")

fig.update_layout(
    plot_bgcolor="white",
    xaxis_showgrid=True,
    yaxis_showgrid=True,
    xaxis_gridcolor="#E5E7E9",
    yaxis_gridcolor="#E5E7E9"
)

fig.show()

print("Population median:", pop_median)
print("MPS median:", mps_median)


Population median: 2477225.0
MPS median: 0.46340445660363183


In [ ]:
#@title 💰 Quadrant Chart —   Revenue vs Population
#-----------------------------------------
# 1. GROUP BY MARKET (to avoid duplicates)
# -----------------------------------------
df_market_summary = (
    df_customer_analysis.groupby(["market_name", "market_state", "population"], as_index=False)
    .agg(
        total_revenue=("total_smiley_revenue", "sum"),
        total_sales=("num_sales", "sum"),
        num_customers=("customer_id", "count")
    )
)

# -----------------------------------------
# 2. Compute medians for quadrant boundaries
# -----------------------------------------
pop_median = df_market_summary["population"].median()
rev_median = df_market_summary["total_revenue"].median()

# -----------------------------------------
# 3. Assign quadrant labels
# -----------------------------------------
def quad(row):
    if row["population"] >= pop_median and row["total_revenue"] >= rev_median:
        return "🏆 High Pop • High Revenue"
    elif row["population"] >= pop_median and row["total_revenue"] < rev_median:
        return "📉 High Pop • Low Revenue"
    elif row["population"] < pop_median and row["total_revenue"] >= rev_median:
        return "💎 Low Pop • High Revenue"
    else:
        return "🪫 Low Pop • Low Revenue"

df_market_summary["quadrant"] = df_market_summary.apply(quad, axis=1)

# -----------------------------------------
# 4. Scatter Plot with Quadrants
# -----------------------------------------
import plotly.express as px
import plotly.graph_objects as go

fig = px.scatter(
    df_market_summary,
    x="population",
    y="total_revenue",
    color="quadrant",
    hover_name="market_name",
    hover_data=["market_state", "total_sales"],
    title="💰 Revenue vs Population — Market Quadrants",
    labels={
        "population": "Market Population",
        "total_revenue": "Total Smiley Revenue ($)"
    },
    color_discrete_map={
        "🏆 High Pop • High Revenue": "#1b9e77",   # green
        "📉 High Pop • Low Revenue": "#d95f02",   # orange
        "💎 Low Pop • High Revenue": "#7570b3",   # purple
        "🪫 Low Pop • Low Revenue": "#aaaaaa"     # gray
    }
)

# Quadrant lines
fig.add_vline(x=pop_median, line_dash="dash", line_color="gray")
fig.add_hline(y=rev_median, line_dash="dash", line_color="gray")

# Style clean + grid
fig.update_layout(
    plot_bgcolor="white",
    xaxis_showgrid=True,
    yaxis_showgrid=True,
    xaxis_gridcolor="#E5E7E9",
    yaxis_gridcolor="#E5E7E9"
)

fig.show()

print("Population median:", pop_median)
print("Revenue median:", rev_median)


Population median: 2477225.0
Revenue median: 1369551.36


In [ ]:
# ----------------------------------------------
#@title 💰 Quadrant Chart: Revenue vs Conversion Rate
# ----------------------------------------------

import plotly.express as px

# Use df_market from your Market Performance Score step
df_q = df_market.copy()

# Compute medians for quadrant boundaries
rev_median = df_q["total_revenue"].median()
conv_median = df_q["conversion_rate"].median()

# Assign quadrants
def quadrant_label(row):
    if row["total_revenue"] >= rev_median and row["conversion_rate"] >= conv_median:
        return "🔼 High Revenue + High Conversion"
    elif row["total_revenue"] >= rev_median and row["conversion_rate"] < conv_median:
        return "💰 High Revenue + Low Conversion"
    elif row["total_revenue"] < rev_median and row["conversion_rate"] >= conv_median:
        return "📈 Low Revenue + High Conversion"
    else:
        return "⚠️ Low Revenue + Low Conversion"

df_q["quadrant"] = df_q.apply(quadrant_label, axis=1)

# Plot quadrant scatter
fig = px.scatter(
    df_q,
    x="total_revenue",
    y="conversion_rate",
    color="quadrant",
    hover_name="market_name",
    hover_data=["market_state"],
    title="📊 Market Quadrants: Revenue vs Conversion Rate",
    labels={
        "total_revenue": "Total Revenue ($)",
        "conversion_rate": "Conversion Rate"
    },
    color_discrete_map={
        "🔼 High Revenue + High Conversion": "#1b9e77",
        "💰 High Revenue + Low Conversion":  "#d95f02",
        "📈 Low Revenue + High Conversion":  "#7570b3",
        "⚠️ Low Revenue + Low Conversion":   "#e7298a",
    }
)

# Add quadrant boundaries
fig.add_vline(x=rev_median, line_dash="dash", line_color="gray")
fig.add_hline(y=conv_median, line_dash="dash", line_color="gray")

# Styling
fig.update_layout(
    plot_bgcolor="white",
    xaxis_showgrid=True,
    yaxis_showgrid=True,
    xaxis_gridcolor="#eee",
    yaxis_gridcolor="#eee"
)

fig.show()

print("Revenue median:", rev_median)
print("Conversion median:", conv_median)


Revenue median: 1369551.3599999996
Conversion median: 0.28318739515280733


In [ ]:
# @title ⭐ Quadrant Chart — Median Sale Price vs Conversion Rate (Value vs Efficiency)

import plotly.express as px

# 1 — Prepare data
df_q = df_market.copy()

x_var = "median_sale"        # Value
y_var = "conversion_rate"    # Efficiency

x_med = df_q[x_var].median()
y_med = df_q[y_var].median()

# 2 — Classify quadrants
def classify_price_eff(row):
    if row[x_var] >= x_med and row[y_var] >= y_med:
        return "💎 High Value • High Efficiency"
    if row[x_var] >= x_med and row[y_var] < y_med:
        return "📈 High Value • Low Efficiency"
    if row[x_var] < x_med and row[y_var] >= y_med:
        return "⚡ Low Value • High Efficiency"
    return "🐢 Low Value • Low Efficiency"

df_q["quadrant"] = df_q.apply(classify_price_eff, axis=1)

# 3 — Friendly color palette
colors = {
    "💎 High Value • High Efficiency": "#0072B2",  # Blue
    "📈 High Value • Low Efficiency": "#56B4E9",  # Sky Blue
    "⚡ Low Value • High Efficiency": "#009E73",  # Green
    "🐢 Low Value • Low Efficiency": "#E69F00",   # Orange
}

# 4 — Plot
fig = px.scatter(
    df_q,
    x=x_var,
    y=y_var,
    color="quadrant",
    hover_name="market_name",
    hover_data=["market_state"],
    color_discrete_map=colors,
    title="⭐ Market Quadrants: Median Sale Price vs Conversion Rate (Value vs Efficiency)",
    labels={x_var: "Median Sale Price ($)", y_var: "Conversion Rate"}
)

fig.add_vline(x=x_med, line_dash="dash", line_color="gray")
fig.add_hline(y=y_med, line_dash="dash", line_color="gray")

fig.update_layout(plot_bgcolor="white")
fig.update_yaxes(showgrid=True, gridcolor="#eee")

fig.show()


In [ ]:
# @title ⚙️ Quadrant Chart — Conversion Rate vs Time to Sale (Speed vs Efficiency)

df_q = df_market.copy()

x_var = "avg_time_to_sale"   # Speed (lower = better)
y_var = "conversion_rate"    # Efficiency

x_med = df_q[x_var].median()
y_med = df_q[y_var].median()

def classify_speed_eff(row):
    if row[x_var] <= x_med and row[y_var] >= y_med:
        return "⚡ Fast & High Efficiency"
    if row[x_var] <= x_med and row[y_var] < y_med:
        return "🚀 Fast but Low Efficiency"
    if row[x_var] > x_med and row[y_var] >= y_med:
        return "🏗️ Slow but Effective"
    return "🐢 Slow & Low Efficiency"

df_q["quadrant"] = df_q.apply(classify_speed_eff, axis=1)

colors = {
    "⚡ Fast & High Efficiency": "#009E73",
    "🚀 Fast but Low Efficiency": "#56B4E9",
    "🏗️ Slow but Effective": "#0072B2",
    "🐢 Slow & Low Efficiency": "#E69F00",
}

fig = px.scatter(
    df_q,
    x=x_var,
    y=y_var,
    color="quadrant",
    hover_name="market_name",
    hover_data=["market_state"],
    color_discrete_map=colors,
    title="⚙️ Market Quadrants: Conversion Rate vs Time to Sale",
    labels={x_var: "Avg Time to Sale (Days)", y_var: "Conversion Rate"}
)

fig.add_vline(x=x_med, line_dash="dash", line_color="gray")
fig.add_hline(y=y_med, line_dash="dash", line_color="gray")

fig.update_layout(plot_bgcolor="white")
fig.update_yaxes(showgrid=True, gridcolor="#eee")
fig.show()


In [ ]:
# @title 🔄 Quadrant Chart — Price Decreases vs Sales Volume (Competition vs Throughput)

df_q = df_market.copy()

x_var = "listings_price_decrease_count"  # Competition / pricing pressure
y_var = "num_sales"                      # Throughput / volume

x_med = df_q[x_var].median()
y_med = df_q[y_var].median()

def classify_comp_through(r):
    if r[x_var] >= x_med and r[y_var] >= y_med:
        return "🔥 High Competition • High Volume"
    if r[x_var] >= x_med and r[y_var] < y_med:
        return "⚠️ High Competition • Low Volume"
    if r[x_var] < x_med and r[y_var] >= y_med:
        return "💼 Low Competition • High Volume"
    return "🐢 Low Competition • Low Volume"

df_q["quadrant"] = df_q.apply(classify_comp_through, axis=1)

colors = {
    "🔥 High Competition • High Volume": "#D55E00",
    "⚠️ High Competition • Low Volume": "#E69F00",
    "💼 Low Competition • High Volume": "#009E73",
    "🐢 Low Competition • Low Volume": "#0072B2",
}

fig = px.scatter(
    df_q,
    x=x_var,
    y=y_var,
    color="quadrant",
    hover_name="market_name",
    hover_data=["market_state"],
    color_discrete_map=colors,
    title="🔄 Market Quadrants: Price Decreases vs Sales Volume",
    labels={x_var: "Price Decreases (30 Days)", y_var: "Sales Volume"}
)

fig.add_vline(x=x_med, line_dash="dash", line_color="gray")
fig.add_hline(y=y_med, line_dash="dash", line_color="gray")

fig.update_layout(plot_bgcolor="white")
fig.update_yaxes(showgrid=True, gridcolor="#eee")

fig.show()


In [ ]:
# @title 🌍 Quadrant Chart — Population vs Sales Volume

import plotly.express as px

df_q2 = df_market.copy()

# ---- Compute medians ----
pop_median = df_q2["population"].median()
sales_median = df_q2["num_sales"].median()

# ---- Assign quadrants ----
def classify_pop_sales(row):
    if row["population"] >= pop_median and row["num_sales"] >= sales_median:
        return "🏙️ Large & High-Sales"
    elif row["population"] >= pop_median and row["num_sales"] < sales_median:
        return "🌆 Large but Low-Sales"
    elif row["population"] < pop_median and row["num_sales"] >= sales_median:
        return "📈 Small but High-Sales"
    else:
        return "🟦 Small & Low-Sales"

df_q2["quadrant"] = df_q2.apply(classify_pop_sales, axis=1)

# ---- Plot ----
fig = px.scatter(
    df_q2,
    x="population",
    y="num_sales",
    color="quadrant",
    hover_name="market_name",
    hover_data=["market_state", "total_revenue"],
    title="🌍 Quadrant Chart — Population vs Sales Volume",
    labels={
        "population": "Market Population",
        "num_sales": "# of Sales"
    },
    color_discrete_map={
        "🏙️ Large & High-Sales": "#E41A1C",
        "🌆 Large but Low-Sales": "#377EB8",
        "📈 Small but High-Sales": "#4DAF4A",
        "🟦 Small & Low-Sales": "#999999"
    }
)

# quadrant lines
fig.add_vline(x=pop_median, line_dash="dash", line_color="gray")
fig.add_hline(y=sales_median, line_dash="dash", line_color="gray")

fig.update_layout(plot_bgcolor="white")
fig.update_yaxes(showgrid=True, gridcolor="#eee")

fig.show()
